In [ ]:
import os
import sys
import re
from tempfile import mkdtemp
import hashlib
import time

print(sys.version_info)

home_dir = os.getenv("HOME")
print(os.getenv("PYTHONPATH"))

import numpy as np
import pickle
from numpy.linalg import svd
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression, ElasticNet, ElasticNetCV, ARDRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics, tree
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Imputer, LabelEncoder
from sklearn.metrics import precision_recall_curve, average_precision_score, brier_score_loss, make_scorer
from sklearn.model_selection import cross_validate, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.externals import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import mstats
import scipy.stats as st
from IPython.display import display, HTML

from tqdm import tqdm_notebook
from fancyimpute import SoftImpute, KNN, MICE
from ehr_utils import *
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import NearMiss
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier, XGBRegressor, plot_tree
from missingpy import MissForest
#import shap
#from predictive_imputer import predictive_imputer

# from keras.models import model_from_json, Sequential
# from keras.layers import Dense, Dropout
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.optimizers import SGD

In [ ]:
#pandas2ri.activate()
%matplotlib inline
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 100)
# colours for plotting for BJA
COLOUR_1 = "#006BFA" # blue
COLOUR_2 = "#378E00" # green
COLOUR_3 = "#E64DFF" # pink
COLOUR_4 = "#FF8000" # gold
COLOUR_5 = "#33FFFF" # light blue
COLOUR_6 = "#FF3300" # orange
PLOT_COLOURS = [COLOUR_1, COLOUR_2, COLOUR_3, COLOUR_4, COLOUR_5, COLOUR_6]
# DPI for figures
fig_dpi = 1200

data_dir="/data/DataNov21_2017"
ccs_icd_mapping_f="/home/blhill/external_data/ccs_dx_icd10cm_2018_1.csv"

# main_f = os.path.join(data_dir, "Main_Data.txt")
# outcomes_f = os.path.join(data_dir, "Outcomes_Data.txt")
# icd_codes_f = os.path.join(data_dir, "ICD9_10Codes.txt")
# main_filtered_f = "Main_Data_filtered.txt"
# main_filtered_working_f = "Main_Data_filtered.working.txt"
asa_predictions_train_f = "paper/asa_imp/imputed_asa_train.txt"
asa_predictions_test_f = "paper/asa_imp/imputed_asa_test.txt"
comorbid_f = "charlson_scores.txt"
pospom_f = "/opt/data/workingdir/blhill/POSPOM_scores.txt"
#main_filtered_f = main_f
main_filtered_f = "/opt/data/workingdir/blhill/main_merged_w_akin_spo2.filtered.main.txt"
test_or_case_id_f = "/opt/data/workingdir/blhill/or_case_id_032018.txt"

# changing the exp_prefix directory below decides which features to load
# and where to save output files
exp_prefix = "preop_no_lab_times"
impute_asa = False
dir_to_save_files = os.path.join("paper/", exp_prefix)
if not os.path.exists(dir_to_save_files):
    os.makedirs(dir_to_save_files)

# set to False to hide IDs    
verbose = False
# default size for plots
default_size=(8,8)
# model type == "regression" || "classification"
model_type = "classification"
# AKIN stage > AKIN_THRESHOLD considered an AKIN_EVENT
AKIN_THRESHOLD=1
# data points measured after this time will be discarded
TIME_THRESHOLD=-60*24*0
# if True, subsample to ensure classes are split 50/50
USE_EQUAL_CLASS_FREQ=False
CLASS_MULTIPLIER=1.0

MIN_ASA_STATUS=1
MAX_ASA_STATUS=5
MIN_AGE=18
MAX_AGE=89

## this variable is the column that we will use as the target variable for the model
TARGET_VARIABLE = 'INPT_DEATH_YN'
#TARGET_VARIABLE = 'AKIN_STAGE1_YN'
#TARGET_VARIABLE = 'MAX_AKIN_STAGE'
#TARGET_VARIABLE = 'AKIN_EVENT'
#TARGET_VARIABLE = 'FLOOR_2_ICU_YN'
#TARGET_VARIABLE = "ASA_STATUS"


df = pd.read_csv(main_filtered_f,sep="|")
print(df.shape)
#display(df.PATIENT_CLASS.unique())
print(len(df.columns))
if verbose:
    display(df.iloc[0:20, :])

def get_sha256_hash(x):
    m = hashlib.sha256()
    m.update(str(x).encode('utf-8'))
    return m.hexdigest().upper()
df["OR_CASE_ID"] = df["OR_CASE_ID"].apply(get_sha256_hash)
    
# make sure binary columns are set to True/False
try:
    df.NITRIC_OXIDE = df.NITRIC_OXIDE.astype(bool)
    df.ART_LINE = df.ART_LINE.astype(bool)
    df.PA_CATHETER = df.PA_CATHETER.astype(bool)
    df.INPT_DEATH_YN = df.INPT_DEATH_YN.astype(bool)
except AttributeError:
    pass
if verbose:
    display(df.describe(include="all"))
ids = df["ADMSN_ID"]
#display(df[ids.isin(ids[ids.duplicated()])].head(30))
# display(df[df["INPT_DEATH_YN"] == True].sort_values(by=["ADMSN_ID", "HRS_ADMSN_TO_SURGERY"]).head(30))

In [ ]:
df.columns.values

In [ ]:
# get methylation sample ID to AtLAs Biobank ID mapping
meth_map_f = "/opt/data/azuremethylationcontainer/2019-9110-1 Halperin Meth Epic/2019-9110 Sample sheet.xlsx"
meth_map = pd.read_excel(meth_map_f, dtype=str)
meth_map["meth_ID"] = meth_map["Chip Barcode ID"] + "_" + meth_map["Stripe ID"]
meth_map["External Sample ID"] = meth_map["External Sample ID"].str.split(".", n=1, expand=True).iloc[:,0]
meth_map.head()
meth_map.describe(include="all")

In [ ]:
# get list of surgeries for testing
test_or_case_ids = pd.read_csv(test_or_case_id_f, header=None)
print(test_or_case_ids.shape)
# get list of patient IDs for testing - we want to remove these from the training set
test_pat_ids = df[df.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0])]["PAT_ID"]
print(test_pat_ids.shape)

In [ ]:
filt_df = pd.read_csv("/opt/data/workingdir/blhill/main_merged_w_akin_spo2.filtered.filtIDX.txt", sep=",")
print(filt_df.ANES_CASE_YN.sum())

In [ ]:
main_unfiltered_f = "/opt/data/workingdir/blhill/main_merged_w_akin_spo2.txt"
df_unfiltered = pd.read_csv(main_unfiltered_f, sep="|")
df_unfiltered["OR_CASE_ID"] = df_unfiltered["OR_CASE_ID"].apply(get_sha256_hash)
df_unfiltered.drop_duplicates(inplace=True)
unfiltered_test_pat_ids = df_unfiltered[df_unfiltered.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0])]["PAT_ID"]
print(unfiltered_test_pat_ids.shape)
print(df_unfiltered.shape)

In [ ]:
print(df_unfiltered.shape)
df_unfiltered = df_unfiltered[df_unfiltered["ANES_CASE_YN"] == 1]
print(df_unfiltered.shape)

In [ ]:
print(df_unfiltered.shape)
df_unfiltered = df_unfiltered[df_unfiltered['LOCATION_GROUP'].isin(['RR OR', 'SM OR'])]
#df_unfiltered = df_unfiltered[(df_unfiltered['LOCATION_GROUP'].str.startswith('RR')) | (df_unfiltered['LOCATION_GROUP'].str.startswith('SM'))]
print(df_unfiltered.shape)

In [ ]:
print(df_unfiltered.shape)
df_unfiltered = df_unfiltered[df_unfiltered['PAT_CLASS'].isin(['INPATIENT', 'SAME DAY ADMIT', 'EMERGENCY', 'OVERNIGHT RECOVERY'])]
print(df_unfiltered.shape)

In [ ]:
print(df_unfiltered.shape)
#df_unfiltered = df_unfiltered[(df_unfiltered["ASA_STATUS"] <= MAX_ASA_STATUS) & (df_unfiltered["ASA_STATUS"] >= MIN_ASA_STATUS)]
df_unfiltered = df_unfiltered[(df_unfiltered["ASA_STATUS"] <= MAX_ASA_STATUS)]
print(df_unfiltered.shape)
df_unfiltered = df_unfiltered[(df_unfiltered["AGE_LT_90"] <= MAX_AGE) & (df_unfiltered["AGE_LT_90"] > MIN_AGE)]
print(df_unfiltered.shape)

In [ ]:
#print_demographics_info(df_unfiltered[~(df_unfiltered.PAT_ID.isin(test_pat_ids))])

In [ ]:
df_unfiltered.PAT_ID = df_unfiltered.PAT_ID.astype(int)
print(df_unfiltered.PAT_ID.dtype)
print(df_unfiltered[~(df_unfiltered.PAT_ID.isin(test_pat_ids))].shape)
print(df_unfiltered[df_unfiltered.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0])].shape)

In [ ]:
# # if a patient dies in an admission, set only the last surgery in the admission to 1
# for name, group in df.groupby("ADMSN_ID"):
#     if group.shape[0] > 1 and group.iloc[-1,:]["INPT_DEATH_YN"] == 1.0:
#         sorted_group = group.sort_values("ADMSN_SURGERY_NUMBER_W_ANES")
#         print(sorted_group)
#         or_case_ids_to_change = sorted_group.iloc[:-1, :]["OR_CASE_ID"]
#         print("="*30)
#         for ocid in or_case_ids_to_change:
#             print(ocid)
#             df.loc[df.OR_CASE_ID == ocid, "INPT_DEATH_YN"] = 0.
#         print("="*30)

In [ ]:
# print df["_merge"].unique()
# print df["AGE_LT_90"].isnull().sum()
# print df[df["_merge"] == 'both'].shape
# print df.INPT_DEATH_YN.mean()
# df2 = pd.read_csv("Main_Data_filtered.txt", sep="\t")
# print df2.shape

# print df.shape
# print df[df.OR_CASE_ID.isin(df2.OR_CASE_ID)].shape
# print df[df.OR_CASE_ID.isin(df2.OR_CASE_ID)].INPT_DEATH_YN.mean()

# df2.drop_duplicates(inplace=True)
# print df2["AGE_LT_89"].isnull().sum()
# # df2.sort_values(by=["ADMSN_ID", "HRS_ADMSN_TO_SURGERY"], inplace=True)
# # df2.drop_duplicates(subset="ADMSN_ID", keep="first", inplace=True)
# # df2 = df2[df2['PAT_CLASS'].isin(['INPATIENT', 'SAME DAY ADMIT', 'EMERGENCY', 'OVERNIGHT RECOVERY'])]
# #df2 = df2[df2['LOCATION_GROUP'].isin(['RR OR', 'SM OR', 'SM SC'])]
# #df2 = df2[(df2["AGE_LT_89"] <= MAX_AGE) & (df2["AGE_LT_89"] >= MIN_AGE)]
# print df2.shape
# print df2.INPT_DEATH_YN.mean()

# outcomes_df = pd.read_csv(os.path.join(data_dir, "Outcomes_Data.txt"), sep='|', header=0)

# old_df = pd.read_csv("Main_Data_filtered.working.txt",sep="\t")
# print "old.df shape:", old_df.shape
# old_df.drop_duplicates(inplace=True)
# print old_df["AGE_LT_90"].isnull().sum()
# #df.drop(['INPT_DEATH_YN'], axis=1, inplace=True)
# # old_df = old_df[old_df["ADMSN_SURGERY_NUMBER"] == 1]
# # old_df = old_df[old_df["ANES_CASE_YN"] == 1]
# #old_df = old_df[(old_df["AGE_LT_90"] <= MAX_AGE) & (old_df["AGE_LT_90"] >= MIN_AGE)]
# # old_df = old_df[old_df['PATIENT_CLASS'].isin(['INPATIENT', 'SAME DAY ADMIT', 'EMERGENCY', 'OVERNIGHT RECOVERY'])]
# print old_df.shape
# old_df = old_df.merge(outcomes_df, on="ADMSN_ID", how="inner")
# print "shape after merging outcomes:", old_df.shape
# print "old df mean:", old_df.INPT_DEATH_YN.mean()

# df2 = df2.merge(old_df[["OR_CASE_ID", "GENDER", "HEIGHT_IN", "WEIGHT_KG", "BMI", "ASA_STATUS", "ADMSN_SURGERY_NUMBER"]], on="OR_CASE_ID", how="inner")
# print "merged shape:", df2.shape
# print df2[~df2.OR_CASE_ID.isin(old_df.OR_CASE_ID)].shape
# print df2[~df2.OR_CASE_ID.isin(old_df.OR_CASE_ID)].INPT_DEATH_YN.mean()

# if verbose:
#     display(df2.head())


In [ ]:
FEATURES_PATH = os.path.join(dir_to_save_files, 'EHR_MAIN_FEATURES.csv')
features_df = pd.read_csv(FEATURES_PATH)

features_dict = {name:list(col.dropna()) for name,col in features_df.items()}
print(features_dict.keys())

final_features = features_dict['final_features']
cat_to_drop = features_dict['cat_to_drop']
outcome_vars = features_dict['outcome_vars']
feat_to_drop = features_dict['feat_to_drop']
cat_vars = features_dict['cat_vars']
contin_vars = features_dict['contin_vars']
bool_outcome_vars = features_dict['bool_outcome_vars']

In [ ]:
# remove HRS_ADMSN_TO_SURGERY from feature list
try:
    final_features.remove('HRS_ADMSN_TO_SURGERY')
    contin_vars.remove('HRS_ADMSN_TO_SURGERY')
    cat_vars.remove('HCUP_CODE')
    cat_vars.remove('PRE_SURG_LOCATION')
except ValueError:
    pass

cat_to_drop.remove("CASE_SRV_NAME")

if 'HRS_ADMSN_TO_SURGERY' not in feat_to_drop:
    feat_to_drop = feat_to_drop.append('HRS_ADMSN_TO_SURGERY')

## Remove duplicate rows

In [ ]:
# remove any people that had multiple surgeries
print(df.shape)
display(df["INPT_DEATH_YN"].mean())
# drop any rows that are exact copies of another row
df.drop_duplicates(inplace=True)
print(df.shape)

In [ ]:
#ids = df["ADMSN_ID"]
# display(df[ids.isin(ids[ids.duplicated()]) & (df["INPT_DEATH_YN"] == True)].sort_values(by=["ADMSN_ID", "HRS_ADMSN_TO_SURGERY"]).head(30))
# display(df[ids.isin(ids[ids.duplicated()]) & (df["INPT_DEATH_YN"] == True)].sort_values(by=["ADMSN_ID", "HRS_ADMSN_TO_SURGERY"]).groupby("ADMSN_ID").apply(lambda x: x.corr()).head(30))
# display(df["INPT_DEATH_YN"].mean())
# display(df[df["INPT_DEATH_YN"] == True & df["HCUP_CODE"].isnull()].head(300))

#display(df[df["INPT_DEATH_YN"] == True].head(300))

# if multiple surgeries for a single ADMSN_ID, take first surgery

In [ ]:
# if multiple surgeries for a single ADMSN_ID, take first surgery
# print df.shape
# df.sort_values(by=["ADMSN_ID", "HRS_ADMSN_TO_SURGERY"], inplace=True)
# df.drop_duplicates(subset="ADMSN_ID", keep="first", inplace=True)
# display(df["INPT_DEATH_YN"].mean())
# print df.shape
# display(df.describe(include="all"))

In [ ]:
print(len(df.CASE_SRV_NAME.unique()))
print(len(df.CASE_SRV_NAME_GROUP.unique()))

## Create outcomes data frame 

In [ ]:
# create data frame for outcomes table
# print outcomes_f
# outcomes_df = pd.read_csv(outcomes_f, sep='|', header=0)
# outcomes_df.INPT_DEATH_YN = outcomes_df.INPT_DEATH_YN.astype(bool)
# outcomes_df.AKIN_STAGE1_YN = outcomes_df.AKIN_STAGE1_YN.apply(lambda x: True if x == 1 else False)
# outcomes_df.AKIN_STAGE2_YN = outcomes_df.AKIN_STAGE2_YN.apply(lambda x: True if x == 1 else False)
# outcomes_df.AKIN_STAGE3_YN = outcomes_df.AKIN_STAGE3_YN.apply(lambda x: True if x == 1 else False)
# outcomes_df.POSTOP_REINTUBATION_YN = outcomes_df.POSTOP_REINTUBATION_YN.astype(bool)
# outcomes_df.ADMSN_TRACH_YN = outcomes_df.ADMSN_TRACH_YN.apply(lambda x: True if x == 1 else False)
# outcomes_df.FLOOR_2_ICU_YN = outcomes_df.FLOOR_2_ICU_YN.astype(bool)
# create new feature AKIN_EVENT, true if MAX_AKIN_STAGE > AKIN_THRESHOLD else false
df['INPT_DEATH_YN'] = df.INPT_DEATH_YN.astype(bool)
df['AKIN_EVENT'] = df['MAX_AKIN_STAGE'].apply(lambda val: True if val > AKIN_THRESHOLD else False)
# display(outcomes_df.describe(include="all"))

## Check frequency of target variables

In [ ]:
# bool_outcome_vars = ['INPT_DEATH_YN', 'AKIN_STAGE1_YN', 'AKIN_STAGE2_YN', 'AKIN_STAGE3_YN', 'POSTOP_REINTUBATION_YN', 
#                      'ADMSN_TRACH_YN', 'FLOOR_2_ICU_YN', 'AKIN_EVENT']

display(df[bool_outcome_vars].mean())

## Join Main data frame with outcomes using ADMSN_ID as key

To add/remove ASA_STATUS from the model, add/remove "ASA_STATUS" from the list of columns below
To get the number of patients in RR OR/SM OR, add "LOCATION_GROUP" to list of columns below 

In [ ]:
# print df.shape
# old_df = pd.read_csv(main_filtered_working_f,sep="\t")
# print old_df.shape
# #df = df.merge(old_df[["ADMSN_ID", "GENDER"]], on="ADMSN_ID", how="inner")
# # removed 
# df = df.merge(old_df[["OR_CASE_ID", "GENDER", "HEIGHT_IN", "WEIGHT_KG", "BMI", "ASA_STATUS", "ADMSN_SURGERY_NUMBER"]], on="OR_CASE_ID", how="inner")
# print df.shape

# Use PRED_ASA_STATUS as a feature

To use PRED_ASA_STATUS as a feature in the model, uncomment out the block of code below

In [ ]:
if "imp_asa" in exp_prefix:
    print("Adding training data predicted ASA")
    asa_train_df = pd.read_csv(asa_predictions_train_f, sep="\t")
    print("asa_train_df shape:", asa_train_df.shape)
    
    print("Adding testing data predicted ASA")
    asa_test_df = pd.read_csv(asa_predictions_test_f, sep="\t")
    print("asa_test_df shape:", asa_test_df.shape)
    
    asa_df = asa_train_df.append(asa_test_df)
    print("asa_df shape:", asa_df.shape)
    
    df = df.merge(asa_df[["OR_CASE_ID", "PRED_ASA_STATUS"]], on="OR_CASE_ID", how="left")
    print(df.shape)
    df.drop_duplicates(inplace=True)
    print(df.shape)

    if verbose:
        df.head(20)

# Use Charlson comorbidity as an input feature

In [ ]:
if exp_prefix == "charlson": 
    print(df.shape)
    comorbid_df = pd.read_csv(comorbid_f, sep="|")
    print(comorbid_df.shape)
    comorbid_df.columns = ["ADMSN_ID", "CHARLSON"]
    comorbid_df = comorbid_df[~(comorbid_df["ADMSN_ID"].isna())]
    comorbid_df["ADMSN_ID"] = comorbid_df["ADMSN_ID"].astype(int)
    print("comorbid_df shape:", comorbid_df.shape)
    comorbid_df.drop_duplicates(inplace=True)
    comorbid_df.drop_duplicates(subset="ADMSN_ID", keep="first", inplace=True)
    print(comorbid_df.head(20))
    print(comorbid_df.dtypes)
    print("comorbid_df shape after dedup:", comorbid_df.shape)
    df["ADMSN_ID"] = df["ADMSN_ID"].astype(int)
    df = df.merge(comorbid_df[["ADMSN_ID", "CHARLSON"]], on="ADMSN_ID", how="left")
    print(df.shape)
    df.drop_duplicates(inplace=True)
    print(df.shape)
    # df.drop(asa_pred_features_to_drop, axis=1, inplace=True)
    # print df.shape
    #df.head(20)
    if verbose:
        df.describe(include="all")

# Use POSPOM score as an input feature

In [ ]:
if exp_prefix == "pospom": 
    print(df.shape)
    pospom_df = pd.read_csv(pospom_f, sep=",", header=0)
    print("pospom_df shape:", pospom_df.shape)
    pospom_df.drop_duplicates(inplace=True)
    # add surgical POSPOM score to diagnosis POSPOM score
    #pospom_df["POSPOM"] = pospom_df["SURGICAL_POSPOM_COMPONENT"] + pospom_df["POSPOM_SCORE"]
    print(pospom_df["SURGICAL_POSPOM_COMPONENT"].isna().mean())
    print(pospom_df["POSPOM_SCORE"].isna().mean())
    
    pospom_df["OR_CASE_ID"] = pospom_df["OR_CASE_ID"].apply(get_sha256_hash)
    
    pospom_df["POSPOM_SCORE"].fillna(0, inplace=True)
    print(pospom_df["POSPOM_SCORE"].isna().mean())
    #pospom_df["POSPOM"] = pospom_df["SURGICAL_POSPOM_COMPONENT"]
    pospom_df["POSPOM"] = pospom_df["SURGICAL_POSPOM_COMPONENT"] + pospom_df["POSPOM_SCORE"]
    #pospom_df.drop(["SURGICAL_POSPOM_COMPONENT", "POSPOM_SCORE"], axis=1, inplace=True)
    pospom_df.drop(["POSPOM_SCORE"], axis=1, inplace=True)
    #pospom_df.drop_duplicates(subset="ADMSN_ID", keep="first", inplace=True)
    print("pospom_df shape after dedup:", pospom_df.shape)
    df = df.merge(pospom_df, on="OR_CASE_ID", how="left")
    print(df.shape)
    df.drop_duplicates(inplace=True)
    print(df.shape)
    print(df["SURGICAL_POSPOM_COMPONENT"].value_counts()/df.shape[0])
    df["SURGICAL_POSPOM_COMPONENT"].hist(bins=20)
    #df["POSPOM"].hist(bins=20)
    plt.show()
    # df.drop(asa_pred_features_to_drop, axis=1, inplace=True)
    # print df.shape
    #df.head(20)
    if verbose:
        df.describe(include="all")

In [ ]:
# # join main data frame with outcomes data
# df.drop(['INPT_DEATH_YN'], axis=1, inplace=True)
# df = df.merge(outcomes_df, on="ADMSN_ID", how="inner")
# print "Column names:", df.columns.values
# print "df shape:", df.shape

# display(df[bool_outcome_vars].mean())
# display(df.describe(include="all"))

## Remove patients with too many missing values

In [ ]:
missing_threshold = 0.3
print(df.shape)
df.dropna(thresh=int(missing_threshold*df.shape[1]), inplace=True)
print(df.shape)

## Check number of null values for each feature

In [ ]:
# get count of null values in each column
print(df.isnull().sum()/df.shape[0])
plt.figure(figsize=(14,5))
plt.bar(np.arange(df.shape[1]), df.isnull().sum()/df.shape[0])
plt.xticks(range(0, len(df.columns.values)), df.columns.values)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('% NaN values')
plt.gca().yaxis.grid(True)
plt.title('Fraction of NaN values in Features')
plt.show()

In [ ]:
print(len(df.HCUP_CODE.unique()))
#print df[(df["ASA_STATUS"] == 2) & (df["INPT_DEATH_YN"] == True)].shape
#df.groupby(["INPT_DEATH_YN","ASA_STATUS"]).hist(layout=(5,1),column="HRS_ADMSN_TO_SURGERY", bins=range(0,100,5))
print(df.shape)
#df = df[df['HRS_ADMSN_TO_SURGERY'] <= 5.]
print(df.shape)
# plt.xlim(2000,4000)

# Filter out surgeries that don't occur in RR or SM operating rooms

In [ ]:
print(df.LOCATION_GROUP.unique())
#df = df[df['LOCATION_GROUP'].isin(['RR OR', 'SM OR','SM SC','SM OB OR','RR OB OR'])]
print(df.shape)
#df = df[df['LOCATION_GROUP'].isin(['RR OR', 'SM OR', 'SM SC'])]
df = df[df['LOCATION_GROUP'].isin(['RR OR', 'SM OR'])]
print(df.shape)

## Filter out surgeries that were not INPATIENT, SAME DAY ADMIT, EMERGENCY, or OVERNIGHT RECOVERY

In [ ]:
# Filter out any surgeries that are not inpatient
#print df['PATIENT_CLASS_INPATIENT']
print("Shape before filtering out outpatient surgeries:", df.shape)
#df = df[df['PATIENT_CLASS'].isin(['INPATIENT', 'SAME DAY ADMIT', 'EMERGENCY', 'OVERNIGHT RECOVERY'])]
df = df[df['PAT_CLASS'].isin(['INPATIENT', 'SAME DAY ADMIT', 'EMERGENCY', 'OVERNIGHT RECOVERY'])]
print("Shape after filtering out outpatient surgeries:", df.shape)

## Filter based on ASA_STATUS, AGE

In [ ]:
try:
    print("Shape before filtering out based on ASA_STATUS:", df.shape)
    print("ASA_STATUS mean:", df.ASA_STATUS.mean())
    df = df[(df["ASA_STATUS"] <= MAX_ASA_STATUS) & (df["ASA_STATUS"] >= MIN_ASA_STATUS)]
    print("Shape after filtering out based on ASA_STATUS:", df.shape)
    print("ASA_STATUS mean:", df.ASA_STATUS.mean())
except AttributeError:
    pass
print("===================================")
print("Mean age:", df.AGE_LT_90.mean())
print("STD age:", df.AGE_LT_90.std())
df = df[(df["AGE_LT_90"] <= MAX_AGE) & (df["AGE_LT_90"] >= MIN_AGE)]
print("Mean age:", df.AGE_LT_90.mean())
print("STD age:", df.AGE_LT_90.std())
print("Shape after filtering out based on AGE_LT_90:", df.shape)

## Check demographic distributions

In [ ]:
print("Number of surgeries in admissions with in-hospital mortality: {}".format(df[(df["INPT_DEATH_YN"]==True) & (df["ADMSN_SURGERY_NUMBER"]==2)].shape))

# df.sort_values(by=["ADMSN_ID", "HRS_ADMSN_TO_SURGERY"], inplace=True)
# df.drop_duplicates(subset="ADMSN_ID", keep="first", inplace=True)
# display(df["INPT_DEATH_YN"].mean())
# print df.shape
# display(df.describe(include="all"))

In [ ]:
#df.groupby("PAT_ID").agg({'ADMSN_ID': 'count', 'OR_CASE_ID': 'count'})

In [ ]:
print("Total number of surgeries:", df.shape[0])
print("Total number of patients:", df["PAT_ID"].unique().shape[0])
print("Total mortalities: {} ({}%)".format(df[df["INPT_DEATH_YN"] == 1].shape[0], float(df[df["INPT_DEATH_YN"] == 1].shape[0])/df.shape[0]*100.))

# df.drop_duplicates(inplace=True)
# df.sort_values(by=["ADMSN_ID", "HRS_ADMSN_TO_SURGERY"], inplace=True)
# df.drop_duplicates(subset="ADMSN_ID", keep="first", inplace=True)
def print_demographics_info(df):
    print("Number of unique patients:", df["PAT_ID"].unique().shape[0])
    print("Number of unique admissions:", df["ADMSN_ID"].unique().shape[0])
    print("Number of surgeries:", df["OR_CASE_ID"].unique().shape[0])
    print("Average number of surgeries per patient:", float(df["OR_CASE_ID"].unique().shape[0])/df["PAT_ID"].unique().shape[0])
    print("Average number of admissions per patient:", float(df["ADMSN_ID"].unique().shape[0])/df["PAT_ID"].unique().shape[0])
    print("Average number of surgeries per admission:", float(df["OR_CASE_ID"].unique().shape[0])/df["ADMSN_ID"].unique().shape[0])
    print("Number of patients with more than one admission: {} ({}%)".format(df.PAT_ID.unique()[df.groupby("PAT_ID")["ADMSN_ID"].nunique() > 1].shape[0], df.PAT_ID.unique()[df.groupby("PAT_ID")["ADMSN_ID"].nunique() > 1].shape[0]/float(df["PAT_ID"].unique().shape[0])*100.))
    print("Number of admissions with more than one surgery: {} ({}%)".format(df.ADMSN_ID.unique()[df["ADMSN_ID"].value_counts() > 1].shape[0], df.ADMSN_ID.unique()[df["ADMSN_ID"].value_counts() > 1].shape[0]/float(df["ADMSN_ID"].unique().shape[0])*100.))
    print("Patients with in-hospital mortality: {} ({}%)".format(df.INPT_DEATH_YN.value_counts()[1], df.INPT_DEATH_YN.value_counts(normalize="True")[1]*100))
    #print("Patients with kidney failure: {} ({}%)".format(df.AKIN_EVENT.value_counts()[1], df.AKIN_EVENT.value_counts(normalize="True")[1]*100))
    print("Mean age:", df.AGE_LT_90.mean(), " std:", df.AGE_LT_90.std())
    print("Number of female patients: {} ({}%)".format(df[df["GENDER"] == "F"].shape[0], df[df["GENDER"] == "F"].shape[0]/float(df.shape[0])*100))

    try:
        print("Number of patients in RR OR: {} ({}%)".format(df.LOCATION_GROUP.value_counts()["RR OR"], df.LOCATION_GROUP.value_counts(normalize="True")["RR OR"]*100))
        print("Number of patients in SM OR: {} ({}%)".format(df.LOCATION_GROUP.value_counts()["SM OR"], df.LOCATION_GROUP.value_counts(normalize="True")["SM OR"]*100))
        #print("Number of patients in SM SC: {} ({}%)".format(df.LOCATION_GROUP.value_counts()["SM SC"], df.LOCATION_GROUP.value_counts(normalize="True")["SM SC"]*100))
    except AttributeError:
        pass
    print("="*40)
    try:
        print("ASA Status:", df.ASA_STATUS.value_counts())
        print("ASA Status (%):", (df.ASA_STATUS.value_counts()/df.shape[0])*100)
    except AttributeError:
        pass

    print("="*40)
    print("Number of mortalities: {} ({}%)".format(df[df["INPT_DEATH_YN"] == 1].shape[0], float(df[df["INPT_DEATH_YN"] == 1].shape[0])/df.shape[0]*100.))
    print("Mean age of mortalities:", df[df["INPT_DEATH_YN"] == 1].AGE_LT_90.mean(), " std:", df[df["INPT_DEATH_YN"] == 1].AGE_LT_90.std())
    print("Number of female mortalities: {} ({}%)".format(df[df["GENDER"] == "F"]["INPT_DEATH_YN"].sum(), 
                                                         df[df["GENDER"] == "F"]["INPT_DEATH_YN"].sum()/float(df["INPT_DEATH_YN"].sum())*100))
    print("Number of male mortalities: {} ({}%)".format(df[df["GENDER"] == "M"]["INPT_DEATH_YN"].sum(), 
                                                         df[df["GENDER"] == "M"]["INPT_DEATH_YN"].sum()/float(df["INPT_DEATH_YN"].sum())*100))
    try:
        print("="*40)
        print("Number of mortalities stratified by location")
        print(df.groupby("LOCATION_GROUP")["INPT_DEATH_YN"].sum())                                
        print(df.groupby("LOCATION_GROUP")["INPT_DEATH_YN"].sum()/float(df["INPT_DEATH_YN"].sum())*100)
    except AttributeError:
        pass
    print("="*40)
    print("Number of mortalities stratified by ASA status")
    print(df.groupby("ASA_STATUS")["INPT_DEATH_YN"].sum())
    print(df.groupby("ASA_STATUS")["INPT_DEATH_YN"].sum()/float(df["INPT_DEATH_YN"].sum())*100)

print("="*40)
print("TRAINING DATA")
print("="*40)
print_demographics_info(df[~(df.PAT_ID.isin(test_pat_ids))])
print("="*40)
print("TESTING DATA")
print("="*40)
print_demographics_info(df[(df.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0]))])
    
if verbose:
    print("="*40)
    print("TRAINING DATA")
    print("="*40)
    print(df[~(df.PAT_ID.isin(test_pat_ids))]["CASE_SRV_NAME"].value_counts())
    print(df[~(df.PAT_ID.isin(test_pat_ids))]["CASE_SRV_NAME"].value_counts()/df[~(df.PAT_ID.isin(test_pat_ids))].shape[0]*100)
    print("="*40)
    print("="*40)
    print("TESTING DATA")
    print("="*40)
    print(df[(df.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0]))]["CASE_SRV_NAME"].value_counts())
    print(df[(df.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0]))]["CASE_SRV_NAME"].value_counts()/df[(df.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0]))].shape[0]*100)
# display(df.describe(include="all"))

In [ ]:
g = df.groupby("CASE_SRV_NAME")
kv = {}
for n, grp in g:
    kv[n] = grp["INPT_DEATH_YN"].sum()
import operator
sorted_x = sorted(kv.items(), key=operator.itemgetter(1), reverse=True)
# for i in sorted_x:
#     print("{}:\t{}".format(i[0], i[1]))
pd.DataFrame(sorted_x, columns=["CASE_SRV_NAME", "INPT_DEATH_COUNT"])

In [ ]:
# df.boxplot(column="PRED_ASA_STATUS", by="ASA_STATUS")
# plt.title("")
# plt.suptitle("")
# plt.ylabel("PRED_ASA_STATUS")

In [ ]:
# look at distribution of HCUP codes in dataset and in patients that die
print("HCUP Codes:") 
hcup_df = pd.DataFrame([df.HCUP_DESC.value_counts(), (df.HCUP_DESC.value_counts()/df.shape[0])*100]).transpose()
hcup_df.columns = ["HCUP_CODE_COUNT", "HCUP_CODE_PERCENT"]
hcup_df.HCUP_CODE_COUNT = hcup_df.HCUP_CODE_COUNT.astype('int32')
hcup_df.HCUP_CODE_PERCENT = hcup_df.HCUP_CODE_PERCENT.round(1)
#display(hcup_df)
hcup_df.to_csv(os.path.join(dir_to_save_files, "HCUP_codes_full_dataset.txt"), sep="\t", header=True, index=True)

hcup_inptdeath_df = pd.DataFrame([df[df["INPT_DEATH_YN"] == True].HCUP_DESC.value_counts(), (df[df["INPT_DEATH_YN"] == True].HCUP_DESC.value_counts()/df[df["INPT_DEATH_YN"] == True].shape[0])*100]).transpose()
hcup_inptdeath_df.columns = ["HCUP_CODE_COUNT", "HCUP_CODE_PERCENT"]
hcup_inptdeath_df.HCUP_CODE_COUNT = hcup_inptdeath_df.HCUP_CODE_COUNT.astype('int32')
hcup_inptdeath_df.HCUP_CODE_PERCENT = hcup_inptdeath_df.HCUP_CODE_PERCENT.round(1)
#display(hcup_inptdeath_df)
hcup_inptdeath_df.to_csv(os.path.join(dir_to_save_files, "HCUP_codes_inpt_death.txt"), sep="\t", header=True, index=True)

df_train = df[~(df.PAT_ID.isin(test_pat_ids))]
hcup_df_train = pd.DataFrame([df_train.HCUP_DESC.value_counts(), (df_train.HCUP_DESC.value_counts()/df_train.shape[0])*100]).transpose()
hcup_df_train.columns = ["HCUP_CODE_COUNT", "HCUP_CODE_PERCENT"]
hcup_df_train.HCUP_CODE_COUNT = hcup_df_train.HCUP_CODE_COUNT.astype('int32')
hcup_df_train.HCUP_CODE_PERCENT = hcup_df_train.HCUP_CODE_PERCENT.round(1)
hcup_df_train["HCUP_CODE_COUNT_TRAIN"] = hcup_df_train.apply(lambda row: "{} ({})".format(row.HCUP_CODE_COUNT, row.HCUP_CODE_PERCENT), axis=1)
#display(hcup_df_train)
hcup_df_train.to_csv(os.path.join(dir_to_save_files, "HCUP_codes_train_data.txt"), sep="\t", header=True, index=True)

df_test = df[df.OR_CASE_ID.isin(test_or_case_ids.iloc[:,0])]
hcup_df_test = pd.DataFrame([df_test.HCUP_DESC.value_counts(), (df_test.HCUP_DESC.value_counts()/df_test.shape[0])*100]).transpose()
hcup_df_test.columns = ["HCUP_CODE_COUNT", "HCUP_CODE_PERCENT"]
hcup_df_test.HCUP_CODE_COUNT = hcup_df_test.HCUP_CODE_COUNT.astype('int32')
hcup_df_test.HCUP_CODE_PERCENT = hcup_df_test.HCUP_CODE_PERCENT.round(1)
hcup_df_test["HCUP_CODE_COUNT_TEST"] = hcup_df_test.apply(lambda row: "{} ({})".format(row.HCUP_CODE_COUNT, row.HCUP_CODE_PERCENT), axis=1)
#display(hcup_df_test)
hcup_df_test.to_csv(os.path.join(dir_to_save_files, "HCUP_codes_test_data.txt"), sep="\t", header=True, index=True)

hcup_df_all = hcup_df_train[["HCUP_CODE_COUNT_TRAIN"]].merge(hcup_df_test[["HCUP_CODE_COUNT_TEST"]], how="left", left_index=True, right_index=True)
hcup_df_all.to_csv(os.path.join(dir_to_save_files, "HCUP_codes_combined_data.txt"), sep="\t", header=True, index=True)
hcup_df_all

In [ ]:
if verbose:
    df[df.HCUP_CODE == 108]

## Remove outlier values
Outliers are values greater than 3 standard deviations from the mean. We only remove outliers for continuous variables (not categorical). 

In [ ]:
string_cols = ['PRE_SURG_LOCATION', 'CASE_SRV_NAME_GROUP', 'CASE_SRV_NAME', 
               'PRIMARY_CPT', 'GENDER', 'HCUP_DESC', 'CPT_DESC', 'PAT_CLASS', 
               'OR_CASE_ID', 'ADMSN_ID']
dff = df.drop(['PRE_SURG_LOCATION', 'CASE_SRV_NAME_GROUP', 'CASE_SRV_NAME', 'PRIMARY_CPT',
                                  'GENDER', 'HCUP_DESC', 'CPT_DESC', 'PAT_CLASS', 'OR_CASE_ID', 'ADMSN_ID'], axis=1)
dff = df.select_dtypes(include=['float64'])
df_string_cols = df[df.columns.difference(dff.columns.values)]
print(df_string_cols.columns.values)
#display(dff.describe())
print (np.abs(st.zscore(dff, axis=1)) > 3)
#print dff.sub(dff.mean()).div(dff.std()).abs().lt(3)
df_no_outliers = dff[dff.sub(dff.mean()).div(dff.std()).abs().lt(4)]
df_no_outliers[df_string_cols.columns.values] = df_string_cols
if verbose:
    display(df_no_outliers.describe(include="all"))
df = df_no_outliers

# Remove variables related to lab times (i.e. *.HRS_2_SURGERY)
If you want to include these variables in the model, comment this block of code

In [ ]:
# # remove variables that have to do with time
# cols_to_keep_no_hrs2surgery = [c for c in df.columns if not c.endswith("HRS_2_SURGERY")]
# print cols_to_keep_no_hrs2surgery
# print len(cols_to_keep_no_hrs2surgery)
# df=df[cols_to_keep_no_hrs2surgery]

# Remove unnecessary features 
These features are either not needed (ex. descriptions of surgery codes) or not wanted

In [ ]:
# save this for checking predictions over time
admsn_surgery_number = df["ADMSN_SURGERY_NUMBER"]
print(admsn_surgery_number.shape)
or_case_id_number = df["OR_CASE_ID"]
admsn_ids = df['ADMSN_ID']
pat_ids = df['PAT_ID']
asa_status = df["ASA_STATUS"]

In [ ]:
# cat_to_drop = ['OR_CASE_ID', 'HCUP_DESC', 'CPT_DESC', 'ECHO_CASE',
#                    'CASE_SRV_NAME', 'CASE_SRV_NAME_GROUP', 'PRIMARY_CPT']
df = df[final_features + [TARGET_VARIABLE]]
#print(df.drop(outcome_vars + cat_to_drop, axis=1).shape[1])
pd.DataFrame(df[final_features].isnull().sum()/df.shape[0]).apply(lambda x: np.round(x, 3)).to_csv(os.path.join(dir_to_save_files, "Feature_List_with_null.txt"), sep="|", header=False, index=True)

In [ ]:
print(df.columns.values)

In [ ]:
df[TARGET_VARIABLE].mean()

## Plot feature correlation 

In [ ]:
import seaborn as sns

sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.zeros_like(df.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16, 13))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df.corr(), mask=mask, cmap=cmap, vmax=1.0, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## One-hot encode categorical variables 

In [ ]:
# one-hot encode values in categorical columns (creates 1 column per possible state)
## removed: 'NITRIC_OXIDE', 'ART_LINE', 'CVC', 'PA_CATHETER',
#categorical_string_cols = ['PAT_CLASS', 'HCUP_CODE', 'GENDER', 'CASE_SRV_NAME', 'PRE_SURG_LOCATION']
#categorical_string_cols = []
# for i in categorical_string_cols:
#     df[i] = pd.Categorical(df[i]).codes
# removed 

In [ ]:
#dummy_cols = ['PAT_CLASS', 'HCUP_CODE', 'GENDER', 'CASE_SRV_NAME', 'PRE_SURG_LOCATION']
#dummy_cols = ['GENDER']
for var in cat_vars:
    try:
        # drop_first uses k-1 dummies out of k categories
        print(var)
        if var not in df.columns.values:
            print("Missing ", var)
        
        df = pd.get_dummies(df, columns=[var], drop_first=True)
        pass
    except ValueError:
        pass
    except KeyError:
        pass
# remove categorical variables (string values)
for var in cat_vars:
    try:
        df.drop(var, axis=1, inplace=True)
        pass
    except ValueError:
        print(var, 'already dropped')
    except KeyError: 
        print(var, 'already dropped')

## Remove features we don't want to include

In [ ]:
for cat in cat_to_drop:
    try:
        df.drop(cat, axis=1, inplace=True)
    except ValueError:
        print(cat, 'already dropped')
    except KeyError:
        print(cat, 'already dropped')

#print df.columns.values
for col in sorted(df.columns.values):
    print(col, "\t\t", df[col].dtype)

## Remove target variables from data frame

In [ ]:
print("Column names:", df.columns.values)

try:
    print(df.shape)
    y = np.ravel(df[TARGET_VARIABLE])
    print(y.mean())
    input_death_yn = df['INPT_DEATH_YN']
    df.drop(TARGET_VARIABLE, axis=1, inplace=True, errors='ignore')
    df.drop(outcome_vars, axis=1, inplace=True, errors='ignore')
except KeyError:
    print(TARGET_VARIABLE, "already dropped")
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html
# default stragegy: mean
# if len(feature_whitelist) > 0:
#     feature_whitelist = [c for c in feature_whitelist if not c.endswith(".HRS_2_SURGERY")]
#     df = df[feature_whitelist]
print(df.isnull().sum())

In [ ]:
# write final feature list to file so we have order of features
pd.DataFrame(df.columns.values).to_csv(os.path.join(dir_to_save_files, exp_prefix + "_final_feature_list.txt"), header=False, index=False)

## Impute missing data

Note that if running with only a single feature (i.e. only ASA status) that you will have to change 
df.iloc[:,0:] to df.iloc[:,1:] since the first column will be the ID and we don't want that

In [ ]:
# This uses mean-value imputation
# imputer = Imputer()
# transformed_X = imputer.fit_transform(df.iloc[:,1:])

In [ ]:
# print df.shape
# [transformed_X, mu_hat, U, s, Vt] = emsvd(df.iloc[:,1:], k=8, tol=1E-3)
#print mu_hat
#df.drop(["ASA_STATUS"], inplace=True)

# try: 
#     imputer = Imputer(strategy="median")
#     #transformed_X = imputer.fit_transform(df.iloc[:,1:])
#     transformed_X = SoftImpute(max_iters=200).complete(df.iloc[:,0:])
#     #transformed_X = MICE().complete(df.iloc[:,1:])
#     #transformed_X = df.iloc[:,1:]
# except ValueError as e:
#     print "ERROR:", e
#     transformed_X = np.array(df.iloc[:,0:])

In [ ]:
#pd.DataFrame(data=transformed_X, columns=df.columns.values[0:]).describe(include="all")

In [ ]:
# flatten y into 1D array
print(len(y))
print("Mean of target vector:", y.mean())
#tdf = pd.DataFrame(data=transformed_X)
#tdf.describe()

In [ ]:
print(y.sum())

In [ ]:
print(or_case_id_number.shape)
print(y.shape)
# Remove patients (and their surgeries) that occured after March 2018
X_train = df[~(pat_ids.isin(test_pat_ids))]
y_train = y[~(pat_ids.isin(test_pat_ids))]
# X_train = df[~(or_case_id_number.isin(test_or_case_ids.iloc[:,0]))]
# y_train = y[~(or_case_id_number.isin(test_or_case_ids.iloc[:,0]))]

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
# use surgeries after March 2018 for testing
X_test = df[or_case_id_number.isin(test_or_case_ids.iloc[:,0])]
y_test = y[or_case_id_number.isin(test_or_case_ids.iloc[:,0])]
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# save IDs for post-processing
X_train_ids = pd.DataFrame()
X_train_ids["OR_CASE_ID"] = or_case_id_number[~(pat_ids.isin(test_pat_ids))]
X_train_ids["ADMSN_ID"] = admsn_ids[~(pat_ids.isin(test_pat_ids))]
X_train_ids["PAT_ID"] = pat_ids[~(pat_ids.isin(test_pat_ids))]
X_train_ids["INPT_DEATH_YN"] = y_train

X_test_ids = pd.DataFrame()
X_test_ids["OR_CASE_ID"] = or_case_id_number[or_case_id_number.isin(test_or_case_ids.iloc[:,0])]
X_test_ids["ADMSN_ID"] = admsn_ids[or_case_id_number.isin(test_or_case_ids.iloc[:,0])]
X_test_ids["PAT_ID"] = pat_ids[or_case_id_number.isin(test_or_case_ids.iloc[:,0])]
X_test_ids["INPT_DEATH_YN"] = y_test

## Split data into training, testing sets

In [ ]:
group_by_admsn = admsn_ids.astype('category').cat.codes
print("number of operations:", len(group_by_admsn))
print("number of unique admission groups:", len(np.unique(group_by_admsn)))
group_kfold = GroupKFold(n_splits=5)

# # take the first fold of the k-fold split
# train_index, test_index = group_kfold.split(np.array(df.iloc[:,0:]), y, group_by_admsn).next()
# #train_index, test_index = group_kfold.split(transformed_X, y, group_by_admsn).next()
# X_train, X_test = df.iloc[train_index], df.iloc[test_index]
# #X_train, X_test = transformed_X[train_index], transformed_X[test_index]
# y_train, y_test = y[train_index], y[test_index]

# X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.2, random_state=43)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
print("num X_train before SMOTE:", len(X_train))
print("num X_test before SMOTE: ", len(X_test))
print("Mean of train target vector:", y_train.mean())
print("Mean of test target vector:", y_test.mean())

# print "NUM null before imputation:", np.count_nonzero(~np.isnan(X_train))
# X_train_fit = predictive_imputer.PredictiveImputer(max_iter=10, initial_strategy='median',
#                                                    f_model="RandomForest").fit(X_train, y_train)
# X_train = X_train_fit.transform(X_train)
# print "NUM null after imputation:", np.count_nonzero(~np.isnan(X_train))
# X_test = X_train_fit.transform(X_test)

#X_train, y_train = SMOTE(kind='borderline1', ratio={True:20000},k_neighbors=13).fit_sample(X_train, y_train)
# use the SMOTE call below
#X_train, y_train = SMOTE(kind='borderline1',k_neighbors=3).fit_sample(X_train, y_train)
#X_train, y_train = NearMiss(random_state=0, version=2).fit_sample(X_train, y_train)

#X_resampled, y_resampled = SMOTE().fit_sample(transformed_X, y)
#X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
print("num X_train:", len(X_train))
print("num X_test: ", len(X_test))
#print "num X_val:", len(X_val)
print("Mean of train target vector:", y_train.mean())
print("Mean of test target vector:", y_test.mean())
# plt.hist(y_train)
# plt.show()
#print "Mean of val target vector:", y_val.mean()

In [ ]:
if verbose:
    display(pd.DataFrame(data=X_train, columns=df.columns.values[0:]).describe(include="all"))

In [ ]:
if verbose:
    pd.DataFrame(data=X_train, columns=df.columns.values[0:]).head(10)

In [ ]:
# try:
#     plt.boxplot(X_train[:,0])
#     print transformed_X[np.where(transformed_X[:,df.columns.get_loc("AGE_LT_89")] < 0),0].mean()
#     print np.max(X_train[:,df.columns.get_loc("AGE_LT_89")])
#     print np.max(X_test[:,df.columns.get_loc("AGE_LT_89")])
#     X_train
#     print X_train.shape
#     print X_test.shape

#     print df.columns.get_loc("AGE_LT_89")
#     print np.mean(transformed_X[:,df.columns.get_loc("AGE_LT_89")])
#     print np.mean(X_train[:,df.columns.get_loc("AGE_LT_89")])
#     print np.mean(X_test[:,df.columns.get_loc("AGE_LT_89")])
#     #X_train[df.columns.get_loc("AGE_LT_89")-1].std()
# except KeyError:
#     pass

## Standardize training, testing data

Here we standardize the training set, and use the mean/std on the test set

In [ ]:
class StandardizeWithNaN(TransformerMixin, BaseEstimator):
    '''This estimator is for standardizing a dataset that has missing data'''
    def __init__(self):
        self.X_mean = []
        self.X_std = []
        pass

    def fit(self, X, y=None):
        # get mean and standard deviation of columns
        self.X_mean = np.nanmean(X, axis=0)
        self.X_std  = np.nanstd(X, axis=0) 
        return self

    def transform(self, X):
        # subtract mean and divide by standard deviation
        return (X - self.X_mean)/self.X_std

In [ ]:
#scaler = StandardScaler()  
scaler = StandardizeWithNaN()
# Don't cheat - fit only on training data
scaler.fit(X_train)
# save scaler object for testing on future data
pickle.dump(scaler, open(os.path.join(dir_to_save_files, "StandardizeWithNaN.pkl"), "wb"))
# apply transformation to training data
X_train = scaler.transform(X_train) 
# apply same transformation to test data
X_test = scaler.transform(X_test) 
#X_val = scaler.transform(X_val)

In [ ]:
class SoftImputeEstimator(TransformerMixin, BaseEstimator):
    '''This estimator is for wrapping the SoftImpute algorithm'''
    def __init__(self, max_iters=200, verbose=True):
        self.max_iters = max_iters
        self.verbose = verbose
        self.fit_count = 0
        self.transform_count = 0
        pass

    def fit(self, X, y=None):
        self.fit_count += 1
        print("SoftImputeEstimator fit count: {}".format(self.fit_count))
        return self

    def transform(self, X):
        self.transform_count += 1
        print("SoftImputeEstimator transform count: {}".format(self.transform_count))
        try:
            # subtract mean and divide by standard deviation
            return SoftImpute(max_iters=self.max_iters, verbose=self.verbose).complete(X.replace(np.inf, np.nan))
        # ValueError raised if no values need to be imputed
        except ValueError:
            return np.array(X)

In [ ]:
X_train.dropna(axis=1, how='all', inplace=True)

In [ ]:
# X_train.drop(["HCUP_CODE", "PRE_SURG_LOCATION"], axis=1, inplace=True)
# X_test.drop(["HCUP_CODE", "PRE_SURG_LOCATION"], axis=1, inplace=True)

In [ ]:
X_train.columns.values

In [ ]:
X_train.dtypes

In [ ]:
print("imputing X_train")
si = Imputer()
si.fit(X_train)
pickle.dump(si, open(os.path.join(dir_to_save_files, "MeanImputer.pkl"), "wb"))

# si = SoftImputeEstimator()
if False:
    coded_cat_vars = []
    for c in cat_vars:
        coded_cat_vars = coded_cat_vars + list(X_train.columns[X_train.columns.str.startswith(c)].values)
    print(coded_cat_vars)
    cat_var_indices = [X_train.columns.get_loc(c) for c in coded_cat_vars]
    print(cat_var_indices)

    si = MissForest(n_estimators=500, max_depth=11, min_samples_leaf=5, verbose=0)
    si.fit(X_train, cat_vars=cat_var_indices)

X_train = si.transform(X_train)

In [ ]:
print("imputing X_test")
#print(np.isnan(X_test).any())
X_test = si.transform(X_test.replace(np.inf, np.nan))

In [ ]:
# if we want to create imputed ASA status, train on training data and 
# generate predicted ASA values for train and test set 
xgb_r = XGBRegressor(max_depth=7, n_estimators=2000, n_jobs=-1)

if impute_asa:
    assert ("asa" not in exp_prefix), "STOP: cannot impute ASA status when it is included as a predictor"
    # get ASA status for train/test cases
    asa_train = asa_status[~(pat_ids.isin(test_pat_ids))]
    asa_test = asa_status[or_case_id_number.isin(test_or_case_ids.iloc[:,0])]
    
    #rfr_pipe = Pipeline([('scaler', scaler), ('impute', imp), ('smt', smt), ('rfr', rfr)])
    #asa_predictions = cross_val_predict(rfr_pipe, df.iloc[:,0:], y, cv=group_kfold.split(df.iloc[:,0:], y, group_by_admsn), n_jobs=1)
    
    t0 = time.time()
    xgb_r.fit(X_train, asa_train)
    t1 = time.time()
    print("Took {} seconds to train.".format(t1-t0))

In [ ]:
if impute_asa:
    from sklearn.metrics import mean_squared_error, r2_score
    imp_asa_train = xgb_r.predict(X_train)
    imp_asa_test = xgb_r.predict(X_test)

    print("---Train ASA Imputation---")
    print("Train MSE:", mean_squared_error(asa_train, imp_asa_train))
    print("Train r^2:", r2_score(asa_train, imp_asa_train))
    print('---Test ASA Imputation---')
    print("Test MSE:", mean_squared_error(asa_test, imp_asa_test))
    print("Train r^2", r2_score(asa_test, imp_asa_test))

In [ ]:
# write predictions to file
if impute_asa:
    X_train_asa = X_train_ids
    X_train_asa["ASA_STATUS"] = asa_train
    X_train_asa["PRED_ASA_STATUS"] = imp_asa_train
    X_train_asa.to_csv(os.path.join(dir_to_save_files, "imputed_asa_train.txt"), sep="\t", header=True, index=False)
    
    X_test_asa = X_test_ids
    X_test_asa["ASA_STATUS"] = asa_test
    X_test_asa["PRED_ASA_STATUS"] = imp_asa_test
    X_test_asa.to_csv(os.path.join(dir_to_save_files, "imputed_asa_test.txt"), sep="\t", header=True, index=False)

In [ ]:
X_train, y_train = SMOTE(kind='borderline1',k_neighbors=3).fit_sample(X_train, y_train)

## Train model(s)
The code is set up to train/test/evaluate any model added to the *models* list. 

To add a model, simply append it to the list, and append a corresponding name 
for the model to the *model_names* list as this will be used for plotting. 

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn import svm

num_threads = -1

models = []
models_cv = []
model_names = []
cachedir = mkdtemp()
#weight_vec = np.multiply((float(X_train.shape[0]) / (2*np.bincount(y))), [1, 1.2])
#print weight_vec

#scaler = StandardScaler() 
scaler = StandardizeWithNaN()
imp = SoftImputeEstimator(verbose=True)
smt = SMOTE(kind='borderline1', k_neighbors=3)
#smt = NearMiss(random_state=0, version=1)
pipeline_stages = [('scaler', scaler), ('impute', imp), ('smt', smt)]
#pipeline_stages = [('scaler', scaler), ('smt', smt)]

## Logistic Regression
#models.append(LogisticRegression(class_weight="balanced"))
if model_type == "classification":
    #models.append(LogisticRegression(class_weight={0:weight_vec[0], 1:weight_vec[1]} ,n_jobs=-1))
    lrg = LogisticRegression(class_weight="balanced", solver="saga", n_jobs=-1)
    models.append(lrg)
    models_cv.append(Pipeline(pipeline_stages + [('lrg', lrg)]))
    model_names.append("Log. Regression")
## ElasticNet
#models.append(ElasticNet(l1_ratio=0.0001, fit_intercept=True))

elasticnet = ElasticNetCV(max_iter=3000, fit_intercept=True, n_jobs=num_threads, cv=5)
elasticnet.fit(X_train, y_train)
en_alpha = elasticnet.alpha_
en_l1_ratio = elasticnet.l1_ratio_
print("ElasticNet alpha:", en_alpha)
print("ElasticNet l1_ratio:", en_l1_ratio)
# model_names.append("ElasticNetCV")

if model_type == "classification":
#     eln = SGDClassifier(loss="log", penalty="elasticnet", alpha=en_alpha, class_weight="balanced",
#                                 l1_ratio=en_l1_ratio, fit_intercept=True, max_iter=3000, n_jobs=num_threads)
    eln = LogisticRegression(penalty="elasticnet", 
                             class_weight="balanced", 
                             l1_ratio=en_l1_ratio,
                             solver="saga",
                             n_jobs=-1)
    models.append(eln)
    models_cv.append(Pipeline(pipeline_stages + [('eln', eln)]))
    model_names.append("ElasticNet")

# if model_type == "regression":
#     models.append(ElasticNet(alpha=en_alpha, l1_ratio=en_l1_ratio, fit_intercept=True))

## Multilayer Perceptron (MLP)
##models.append(MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(32, 4), early_stopping=True))
#models.append(MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 6), early_stopping=True))
#model_names.append("MLP")

## Random Forest 
if model_type == "classification":
    rfc = RandomForestClassifier(class_weight="balanced", 
                                 oob_score=True, 
                                 n_estimators=2000, 
                                 n_jobs=num_threads, 
                                 max_features="log2",
                                 max_depth=7, 
                                 min_samples_leaf=3)
#    rfc = RandomForestClassifier(class_weight={1:1.0, 2:1.0, 3:1.1, 4:1.3, 5:1.5}, oob_score=True, n_estimators=2000, n_jobs=-1, max_features="log2")
    models.append(rfc)
    models_cv.append(Pipeline(pipeline_stages + [('rfc', rfc)]))
    model_names.append("Random Forest")
    
    xgb = XGBClassifier(max_depth=5, objective="binary:logistic", n_estimators=2000, n_jobs=num_threads, silent=True)
    models.append(xgb)
    models_cv.append(Pipeline(pipeline_stages + [('xgb', xgb)]))
    model_names.append("XGBClassifier")
    
if model_type == "regression":
    models.append(RandomForestRegressor(max_depth=5, oob_score=True, n_estimators=2000, n_jobs=num_threads, max_features="log2"))
    model_names.append("RanfomForestRegressor")
    
    xgb = XGBRegressor(max_depth=7, n_estimators=2000, n_jobs=num_threads)
    models.append(xgb)
    model_names.append("XGBRegressor")
# models.append(GradientBoostingClassifier(n_estimators=1000))
# model_names.append("GradientBoostedClass")



In [ ]:
sklearn.__version__

In [ ]:
# fit the models
for mod in models:
    t0 = time.time()
    mod.fit(X_train, y_train) 
    t1 = time.time()
    print("Took {} seconds to train.".format(t1-t0))

In [ ]:
# dump models to file
for i in range(len(models)):
    pickled_model_name = "{}_train_sk{}.pkl".format(model_names[i], sklearn.__version__)
    pickle.dump(models[i], open(os.path.join(dir_to_save_files, pickled_model_name), 'wb'))

## Examine feature relevance

In [ ]:
# import xgboost
# shap.initjs()
# #print df.columns.values
# #xdf = pd.DataFrame(data=X_train, columns=df.columns.values[1:])
# #bst = xgboost.train({"learning_rate": 0.01}, xgboost.DMatrix(xdf, label=y_train), 100)
# #shap_values = bst.predict(xgboost.DMatrix(xdf), pred_contribs=True)
# #shap.force_plot(shap_values[0], xdf[0,:])
# print(type(models[2]))
# shap_values = shap.TreeExplainer(models[2]).shap_values(X_train[1:500, :])
# print shap_values.shape

In [ ]:
# print len(shap_values[1])

In [ ]:
# shap.summary_plot(shap_values, X_train[1:500, :])

In [ ]:
feature_importance_df = pd.DataFrame()
feature_importance_df["Features"] = df.columns.values
print(feature_importance_df.shape)

for i in range(len(models)):
    try:
        
        feature_importance_df[model_names[i]] = models[i].feature_importances_
    except AttributeError:
        #print(models[i].coef_[0].shape)
        feature_importance_df[model_names[i]] = models[i].coef_[0]
feature_importance_df = feature_importance_df[["Features", "Log. Regression", "ElasticNet", "Random Forest", "XGBClassifier"]]
feature_importance_df = feature_importance_df.sort_values("Random Forest", ascending=False)
feature_importance_df.to_csv(os.path.join(dir_to_save_files, "model_feature_importance.txt"), sep=",", header=True, index=False)

In [ ]:
for i in range(len(models)):
    if model_names[i] == 'Random Forest':
        print("Random Forest Feature Importances:")
        display(pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].feature_importances_))).sort_values([1], ascending=False))
        pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].feature_importances_))).sort_values([1], ascending=False).to_csv(os.path.join(dir_to_save_files, "random_forest_feature_importance.txt"), sep="|", index=False,header=False)
    elif model_names[i] == 'Log. Regression':
        print("Log. Regression Coef. Weights:")
        display(pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].coef_[0]))).sort_values([1], ascending=False))
        pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].coef_[0]))).sort_values([1], ascending=False).to_csv(os.path.join(dir_to_save_files, "log_reg_feature_importance.txt"), sep="|", index=False,header=False)
    elif model_names[i] == 'ElasticNet':
        print("ElasticNet Coef. Weights:")
        display(pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].coef_[0]))).sort_values([1], ascending=False)) 
        pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].coef_[0]))).sort_values([1], ascending=False).to_csv(os.path.join(dir_to_save_files, "elasticnet_feature_importance.txt"), sep="|", index=False,header=False)
    elif model_names[i] == 'XGBClassifier':
        print("XGBoost Feature Importances:")
        display(pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].feature_importances_))).sort_values([1], ascending=False))
        pd.DataFrame(zip(df.columns.values[0:], np.transpose(models[i].feature_importances_))).sort_values([1], ascending=False).to_csv(os.path.join(dir_to_save_files, "xgboost_feature_importance.txt"), sep="|", index=False,header=False)

In [ ]:
## Create bar plots for feature importance
# fig, ax = plt.subplots(figsize=(10,70))
# eldf = zip(df.columns.values[1:], np.transpose(models[model_names.index("ElasticNet")].coef_[0]))
# eldf = sorted(eldf, key=lambda x: x[1], reverse=False)

# ax.barh(range(len(eldf)), [x[1] for x in eldf])
# ax.set_yticks(np.arange(len(eldf)))
# ax.set_yticklabels([x[0] for x in eldf])
# ax.set_xlabel("Model Weight")
# plt.gca().yaxis.grid(True)
# plt.tight_layout()
# plt.savefig('elasticnet_feature_importance_plot.png', bbox_inches='tight')

## Generate class probabilities

In [ ]:
def calibrate_predicted_probability(pred_prob, p_1):
#def calibrate_predicted_probability(pred_prob, orig_freq, oversampled_freq):
    if pred_prob == 0.0: 
        return 0.0
    p_0 = 1.0 - p_1
    #return 1./(1.+((1./orig_freq)-1.)/((1./oversampled_freq)-1.)*((1./pred_prob)-1.))
    # formula below from Christine Lee's paper
    return 1.0/(1.0+((1.0/pred_prob)-1.0)*(p_0/p_1))

calibrate_predicted_probability_vec = np.vectorize(calibrate_predicted_probability)

In [ ]:
#print calibrate_predicted_probability_vec([0.09500048], y_test.mean(), y_train.mean())

def generate_class_probs(models, X_test):
    try:
        # generate class probabilities
        #model_probs = [calibrate_predicted_probability_vec(mod.predict_proba(X_test), y_test.mean()) for mod in models]
        model_probs = [mod.predict_proba(X_test) for mod in models]
        #probs = model2.predict(X_test)
        #print model_probs
    except AttributeError as e:
        print(e)
        print(mod)
        pass
    return model_probs

model_probs = generate_class_probs(models, np.array(X_test))

In [ ]:
# death_indices = [i for i,x in enumerate(y_test) if x]
# print model_probs[2][death_indices][:,1]
# plt.hist(model_probs[2][:,1], bins=100, density=True)

## Predict class labels & plot mean square error

In [ ]:
# predict class labels for the test set
def predict_given_threshold(probs, threshold=0.15):
    return [True if x[1] > threshold else False for x in probs]

#model_predictions = [np.rint(mod.predict(X_test)) for mod in models]
#model_predictions = [predict_given_threshold(probs, y_test.mean()) for probs in model_probs]
model_predictions = [mod.predict(np.array(X_test)) for mod in models]
print(model_predictions[0][0:10])
print(model_probs[0][0:10])
print(y_test[0:10])
#print np.rint(model_predictions[0][0:10])
#mse = [metrics.mean_squared_error(y_test, predicted) for predicted in model_predictions]
#print [x[1] for x in model_probs[0]]

# def plot_mean_square_error(models, model_names, mse):
#     print "Mean Square Error:"
#     for i in range(len(models)):
#         print model_names[i],"\t\t", mse[i]

#     plt.figure(figsize=default_size)
#     plt.gca().yaxis.grid(True)
#     plt.bar(np.arange(len(models)), mse, align='center')
#     plt.xticks(np.arange(len(models)), model_names)
#     plt.xlabel('Classifier')
#     plt.ylabel('Mean Square Error (MSE)')
#     plt.title('Classifier MSE')
#     plt.legend(loc="lower right")
#     plt.show()
    
#plot_mean_square_error(models, model_names, mse)
#print metrics.mean_squared_error(y_test, [x[1] for x in model_probs[0]])

## Calculate accuracy and AUC

In [ ]:
plot_accuracy_roc_auc(models, model_names, model_predictions, model_probs, y_train, y_test, "accuracy_roc_auc.png")

## Plot ROC curve

In [ ]:
plot_roc_curve(models, model_names, model_probs, y_test, os.path.join(dir_to_save_files,"roc_curve.png"))

In [ ]:
confusion_matrix_classification_rpt(models, model_names, model_predictions, y_test, dir_to_save_files)

## Plot precision-recall curve

In [ ]:
plot_precision_recall_curve(models, model_names, model_probs, y_test, os.path.join(dir_to_save_files, "precision_recall_curve.png"))

In [ ]:
def ranking_precision_score(y_true, y_pred, k=10):
    """Precision at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    y_pred : array-like, shape = [n_samples]
        Predicted scores.
    k : int
        Rank.
    Returns
    -------
    precision @k : float
    """
    unique_y = np.unique(y_true)

    if len(unique_y) > 2:
        raise ValueError("Only supported for two relevance levels.")

    pos_label = unique_y[1]
    n_pos = np.sum(y_true == pos_label)

    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])
    n_relevant = np.sum(y_true == pos_label)

    # Divide by min(n_pos, k) such that the best achievable score is always 1.0.
    return float(n_relevant) / min(n_pos, k)


def plot_precision_at_k(models, model_names, model_probs, y_test, filename):
    plt.figure(figsize=default_size)
    lw = 2
    for i in range(len(model_names)):
        k_vals = np.arange(2, 100)
        if isinstance(y_test, list):
            precision_at_k = [ranking_precision_score(np.array(y_test[i]), model_probs[i][:,1], k=k) for k in k_vals]
        else:
            precision_at_k = [ranking_precision_score(np.array(y_test), model_probs[i][:,1], k=k) for k in k_vals]
        #plt.plot(fpr, tpr, PLOT_COLOURS[i], lw=lw, label=model_names[i]+' (AUC = %0.4f)' % roc_auc)
        plt.plot(k_vals, precision_at_k, PLOT_COLOURS[i], lw=lw, label=model_names[i])
    #plt.plot(fpr, tpr, color='darkorange',
    #         lw=lw, label='ROC curve (area = %0.4f)' % roc_auc)
#    plt.xlim([0.0, 1.05])
    plt.ylim([0.0, 1.05])
    plt.yticks(np.arange(0., 1.1, 0.1))
#    plt.xticks(np.arange(0., 1.1, 0.1))
    ax = plt.axes()
    # default width = 2, def length = 6
    ax.set_yticks(np.arange(0., 1.1, 0.1), minor=True)
#    ax.set_xticks(np.arange(0., 1.1, 0.1), minor=True)
    ax.tick_params(direction='out', length=6, width=0.25, colors='black',labelsize=label_text_size)
    ax.tick_params(axis = 'both', which = 'minor', width=0.25)
    plt.xlabel('k', fontsize=label_text_size)
    plt.ylabel('Precision', fontsize=label_text_size)
    #plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right",fontsize=legend_text_size)
    plt.tight_layout()
    plt.savefig(filename, format="tif", dpi=fig_dpi)
    plt.show()

plot_precision_at_k(models, model_names, model_probs, y_test, os.path.join(dir_to_save_files, "precision_at_k.png"))

## Plot calibration curves

In [ ]:
# plot derived from: http://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html#sphx-glr-auto-examples-calibration-plot-compare-calibration-py
plt.figure(figsize=(10, 10))
ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
ax2 = plt.subplot2grid((3, 1), (2, 0))

ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
for i in range(len(models)):
    #clf.fit(X_train, y_train)
    #if hasattr(models[i], "predict_proba"):
    #    prob_pos = clf.predict_proba(X_test)[:, 1]
    #else:  # use decision function
        #prob_pos = clf.decision_function(X_test)
        #prob_pos = \
        #    (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
    calibrated_model_probs = [calibrate_predicted_probability(j, y_test.mean()) for j in model_probs[i][:, 1]]
    print(model_probs[i][:, 1][0:7])
    print(calibrated_model_probs[0:7])
    #calibrated_model_probs = calibrate_predicted_probability_vec(model_probs[i][:, 1], y_test.mean(), y_train.mean())
    fraction_of_positives, mean_predicted_value = \
        calibration_curve(y_test, calibrated_model_probs, n_bins=10)

    ax1.plot(mean_predicted_value, fraction_of_positives, PLOT_COLOURS[i], "s-",
             label="%s" % (model_names[i], ))

    ax2.hist(model_probs[i], range=(0, 1), bins=10, label=model_names[i],
             histtype="step", lw=2)

ax1.set_ylabel("Fraction of positives")
ax1.set_ylim([-0.05, 1.05])
ax1.legend(loc="upper left")
ax1.set_title('Calibration plots  (reliability curve)')

ax2.set_xlabel("Mean predicted value")
ax2.set_ylabel("Count")
ax2.legend(loc="upper center", ncol=2)

plt.tight_layout()
plt.savefig(os.path.join(dir_to_save_files, "calibration_plots.png"), format="png", dpi=fig_dpi)
plt.show()

# Calculate Brier Score Loss

In [ ]:
print("Brier Score Loss:")
for i in range(len(models)):
    
    brier_loss = brier_score_loss(y_test, model_probs[i][:, 1])
    print(model_names[i],"\t\t", brier_loss)
    

# Write predicted probabilities to file

In [ ]:
for i in range(len(models)):
    print(model_names[i])
    X_test_ids[model_names[i] + "_probs"] = model_probs[i][:,1]
    X_test_ids[model_names[i] + "_preds"] = model_predictions[i]
X_test_ids.to_csv(os.path.join(dir_to_save_files, "model_pred_probs_v3.txt"), sep="\t", header=True, index=False)
if verbose:
    X_test_ids.head()

# Bootstrap the predicted probabilities, grouping by patient to preserve correlation structure

1. Until we have _n_ predictions, sample patients (with replacement) and keep count of how many total predictions we've sampled
2. Repeat (1) for 1000 trials, so that we have 1000 _n_ predictions
3. For each of the 1000 trials, compute the metrics of interest
4. For each metric of interest, sort the values and take the 25th and 975th values as the lower and upper confidence intervals. Take the mean of all values to get the bootstrapped mean. 

In [ ]:
print(X_test_ids.columns.values)
X_test_ids.set_index(keys="PAT_ID", inplace=True, drop=False)
X_test_ids = X_test_ids.sort_index()
print(X_test_ids.shape)

In [ ]:
from tqdm import tqdm_notebook
import time
n = X_test_ids.shape[0]
#n = 100
print("n:", n)

bootstrap_samples = pd.DataFrame()
trials = 100

# cache for performance
unique_pat_ids = X_test_ids["PAT_ID"].unique()

for trial in range(trials):
    bootstrap_trial = []
    # continue until we have n predictions for this trial
    i = 0
    num_surg = 0
    #while bootstrap_trial.shape[0] < n:
    while num_surg < n:
        t0 = time.time()
        # get patient from list of unique patients
        sample_patient_id = np.random.choice(unique_pat_ids)
        t1 = time.time()
        sample_preds = X_test_ids[X_test_ids.PAT_ID.isin([sample_patient_id])]
#        sample_preds = X_test_ids.loc[[sample_patient_id]]
#        print(sample_preds.values)
#        print(sample_preds.values.shape[0])
        t2 = time.time()
#        bootstrap_trial.append(sample_preds.values)
        for p in sample_preds.values:
#             print(p)
            bootstrap_trial.append(p)
#         print("append:", time.time()-t2)
#         print("random choice:", t1-t0)
#         print("data access:", t2-t1)
        i += 1
        num_surg += sample_preds.shape[0]
    bootstrap_trial = pd.DataFrame.from_records(bootstrap_trial, columns=X_test_ids.columns)
    bootstrap_trial["trial_num"] = trial
    print(bootstrap_trial.shape)
    bootstrap_samples = bootstrap_samples.append(bootstrap_trial)
bootstrap_samples.head()

In [ ]:
# calculate model performance metrics and save to file

def tp(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[1, 1]
def tn(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[1, 0]

def ci(results, alpha=0.05):
    results = sorted(results)
    max_index = int(np.floor((1.-alpha/2.)*len(results)))
    min_index = int(np.floor((alpha/2.)*len(results)))
    #print(len(results))
    #print("min index:", min_index)
    #print("max_index:", max_index)
    return results[min_index], results[max_index]

metrics_dict = {}
metric_string = "{:0.3f} ({:0.3f}-{:0.3f})"

for i in range(len(model_names)):
    metrics_dict[model_names[i]] = []
    
    model_probs = bootstrap_samples.groupby("trial_num")[model_names[i] + "_probs"].apply(list)
    model_preds = bootstrap_samples.groupby("trial_num")[model_names[i] + "_preds"].apply(list)
    y_test = bootstrap_samples.groupby("trial_num")["INPT_DEATH_YN"].apply(list)
    
#     model_probs = model_probs_cv[i]
#     model_preds = model_preds_cv[i]
#     y_test = testing_labels_cv[i]
    
    tp_vec = [tp(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    tn_vec = [tn(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    fp_vec = [fp(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    fn_vec = [fn(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    
    fpr_vec = np.array([metrics.roc_curve(y_true, probs)[0] for (y_true, probs) in zip(y_test, model_probs)])
    tpr_vec = np.array([metrics.roc_curve(y_true, probs)[1] for (y_true, probs) in zip(y_test, model_probs)])
    roc_auc_vec = np.array([metrics.auc(fpr, tpr) for (fpr, tpr) in zip(fpr_vec, tpr_vec)])
    precision_vec = np.array([metrics.precision_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    recall_vec = np.array([metrics.recall_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    f1_vec = np.array([metrics.f1_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    accuracy_vec = np.array([metrics.accuracy_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    specificity_vec = np.array([(float(t_n)/(t_n + f_p)) for (t_n, f_p) in zip(tn_vec, fp_vec)])
    brier_score_vec = np.array([brier_score_loss(y_true, probs) for (y_true, probs) in zip(y_test, model_probs)])
    print("==================================================")
    print("MODEL: ", model_names[i])
    print("ROC AUC:\t", metric_string.format(roc_auc_vec.mean(), ci(roc_auc_vec)[0], ci(roc_auc_vec)[1]))
    print("specificity:\t", metric_string.format(specificity_vec.mean(), ci(specificity_vec)[0], ci(specificity_vec)[1]))
    print("precision:\t", metric_string.format(precision_vec.mean(), ci(precision_vec)[0], ci(precision_vec)[1]))
    print("recall:\t\t", metric_string.format(recall_vec.mean(), ci(recall_vec)[0], ci(recall_vec)[1]))
    print("f1 score:\t", metric_string.format(f1_vec.mean(), ci(f1_vec)[0], ci(f1_vec)[1]))
    print("accuracy:\t", metric_string.format(accuracy_vec.mean(), ci(accuracy_vec)[0], ci(accuracy_vec)[1]))
    print("brier score:\t", metric_string.format(brier_score_vec.mean(), ci(brier_score_vec)[0], ci(brier_score_vec)[1]))
    
    metrics_dict[model_names[i]].append(metric_string.format(roc_auc_vec.mean(), ci(roc_auc_vec)[0], ci(roc_auc_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(specificity_vec.mean(), ci(specificity_vec)[0], ci(specificity_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(precision_vec.mean(), ci(precision_vec)[0], ci(precision_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(recall_vec.mean(), ci(recall_vec)[0], ci(recall_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(f1_vec.mean(), ci(f1_vec)[0], ci(f1_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(accuracy_vec.mean(), ci(accuracy_vec)[0], ci(accuracy_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(brier_score_vec.mean(), ci(brier_score_vec)[0], ci(brier_score_vec)[1]))
    
metrics_df = pd.DataFrame.from_dict(metrics_dict, orient="index")
metrics_df.columns =["ROC AUC", "Specificity", "Precision", "Recall", "F1 Score", "Accuracy", "Brier Score"]
metrics_df = metrics_df[["Accuracy", "F1 Score", "Precision", "Recall", "Specificity", "ROC AUC", "Brier Score"]]
metrics_df = metrics_df.transpose()
metrics_df = metrics_df[["Log. Regression", "ElasticNet", "Random Forest", "XGBClassifier"]]
metrics_df.to_csv(os.path.join(dir_to_save_files, "model_scores_manual_cv.txt"), sep=",", header=True, index=True)
metrics_df.head()

In [ ]:
foo bar baz

## Cross Validation of Model Metrics

In [ ]:
num_folds = 5

group_by_admsn = admsn_ids.astype('category').cat.codes
print "number of operations:", len(group_by_admsn)
print "number of unique admission groups:", len(np.unique(group_by_admsn))
group_kfold = GroupKFold(n_splits=num_folds)

# save GroupKFold object as pickle object
pickle.dump(group_kfold, open( os.path.join(dir_to_save_files, "group_kfold.p"), "wb" ))

In [ ]:
# keep track of these during cross-validation
model_probs_cv = []
model_preds_cv = []
testing_labels_cv = []
admsn_ids_cv = []
or_case_ids_cv = []
test_index_cv = []
admsn_surgery_num_cv = []
for model in model_names:
    model_probs_cv.append([])
    model_preds_cv.append([])
    testing_labels_cv.append([])

cv_num = 1
for train_index, test_index in group_kfold.split(np.array(df.iloc[:,0:]), y, group_by_admsn):
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # keep track of these for writing to file
    admsn_ids_cv = admsn_ids_cv + list(admsn_ids.iloc[test_index])
    or_case_ids_cv = or_case_ids_cv + list(or_case_id_number.iloc[test_index])
    admsn_surgery_num_cv = admsn_surgery_num_cv + list(admsn_surgery_number.iloc[test_index])
    test_index_cv = test_index_cv + [cv_num] * len(list(test_index))
    
    print "admsn_ids_cv len:", len(admsn_ids_cv)
    print "or_case_ids_cv len:", len(or_case_ids_cv)
    print "admsn_surgery_num_cv len:", len(admsn_surgery_num_cv)
    print "test_index_cv len:", len(test_index_cv)
    
    # scale the training and testing data
    scaler = StandardizeWithNaN()
    # fit on training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train) 
    # apply same transformation to test data
    X_test = scaler.transform(X_test) 
    
    print("imputing X_train")
    si = SoftImputeEstimator(verbose=False)
    X_train = si.transform(X_train)
    print("imputing X_test")
    X_test = si.transform(X_test.replace(np.inf, np.nan))
    
    # oversample the training dataset
    X_train, y_train = SMOTE(kind='borderline1',k_neighbors=3).fit_sample(X_train, y_train)
    
    i = 0
    for mod in models:
        print("fitting {} model".format(model_names[i]))
        mod.fit(X_train, y_train)
        print("finished fitting...")
        model_probs = mod.predict_proba(np.array(X_test))
        model_predictions = mod.predict(np.array(X_test))
        model_probs_cv[i].append(model_probs)
        model_preds_cv[i].append(model_predictions)
        testing_labels_cv[i].append(y_test)
        i += 1
    cv_num += 1
        

In [ ]:
# calculate model performance metrics and save to file

def tp(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[1, 1]
def tn(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[1, 0]

def ci(results):
    return st.t.interval(0.95, len(results)-1, loc=np.mean(results), scale=st.sem(results))

metrics_dict = {}
metric_string = "{:0.3f} ({:0.3f}-{:0.3f})"

for i in range(len(model_names)):
    metrics_dict[model_names[i]] = []
    
    model_probs = model_probs_cv[i]
    model_preds = model_preds_cv[i]
    y_test = testing_labels_cv[i]
    
    tp_vec = [tp(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    tn_vec = [tn(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    fp_vec = [fp(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    fn_vec = [fn(y_true, y_pred) for (y_true, y_pred) in zip(y_test, model_preds)]
    
    fpr_vec = np.array([metrics.roc_curve(y_true, probs[:, 1])[0] for (y_true, probs) in zip(y_test, model_probs)])
    tpr_vec = np.array([metrics.roc_curve(y_true, probs[:, 1])[1] for (y_true, probs) in zip(y_test, model_probs)])
    roc_auc_vec = np.array([metrics.auc(fpr, tpr) for (fpr, tpr) in zip(fpr_vec, tpr_vec)])
    precision_vec = np.array([metrics.precision_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    recall_vec = np.array([metrics.recall_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    f1_vec = np.array([metrics.f1_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    accuracy_vec = np.array([metrics.accuracy_score(y_true, y_pred) for (y_true, y_pred) in zip (y_test, model_preds)])
    specificity_vec = np.array([(float(t_n)/(t_n + f_p)) for (t_n, f_p) in zip(tn_vec, fp_vec)])
    print "=================================================="
    print "MODEL: ", model_names[i]
    print "ROC AUC:\t", metric_string.format(roc_auc_vec.mean(), ci(roc_auc_vec)[0], ci(roc_auc_vec)[1])
    print "specificity:\t", metric_string.format(specificity_vec.mean(), ci(specificity_vec)[0], ci(specificity_vec)[1])
    print "precision:\t", metric_string.format(precision_vec.mean(), ci(precision_vec)[0], ci(precision_vec)[1])
    print "recall:\t\t", metric_string.format(recall_vec.mean(), ci(recall_vec)[0], ci(recall_vec)[1])
    print "f1 score:\t", metric_string.format(f1_vec.mean(), ci(f1_vec)[0], ci(f1_vec)[1])
    print "accuracy:\t", metric_string.format(accuracy_vec.mean(), ci(accuracy_vec)[0], ci(accuracy_vec)[1])
    
    metrics_dict[model_names[i]].append(metric_string.format(roc_auc_vec.mean(), ci(roc_auc_vec)[0], ci(roc_auc_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(specificity_vec.mean(), ci(specificity_vec)[0], ci(specificity_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(precision_vec.mean(), ci(precision_vec)[0], ci(precision_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(recall_vec.mean(), ci(recall_vec)[0], ci(recall_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(f1_vec.mean(), ci(f1_vec)[0], ci(f1_vec)[1]))
    metrics_dict[model_names[i]].append(metric_string.format(accuracy_vec.mean(), ci(accuracy_vec)[0], ci(accuracy_vec)[1]))
    
metrics_df = pd.DataFrame.from_dict(metrics_dict, orient="index")
metrics_df.columns =["ROC AUC", "Specificity", "Precision", "Recall", "F1 Score", "Accuracy"]
metrics_df.head()
metrics_df.to_csv(os.path.join(dir_to_save_files, "model_scores_manual_cv.txt"), sep=",", header=True, index=True)

In [ ]:
model_probs_dict = {"OR_CASE_ID": or_case_ids_cv,
                    "ADMSN_ID": admsn_ids_cv, 
                    "ADMSN_SURGERY_NUMBER": admsn_surgery_num_cv, 
                    "test_index": test_index_cv,
                    "INPT_DEATH_YN": []}

for model_name in model_names:
    model_probs_dict[model_name] = []

for j in range(group_kfold.get_n_splits()):
    for i in range(len(model_names)):
        print(len(model_probs_cv[i][j]))
        model_probs_dict[model_names[i]] = model_probs_dict[model_names[i]] + list(model_probs_cv[i][j][:,1])
        if i == 0:
            model_probs_dict["INPT_DEATH_YN"] = model_probs_dict["INPT_DEATH_YN"] + list(testing_labels_cv[i][j])
            
for k,v in model_probs_dict.iteritems():
    print k, len(v)
            
model_probs_df = pd.DataFrame(data=model_probs_dict)
model_probs_df.to_csv(os.path.join(dir_to_save_files, "model_pred_probs_v2.txt"), sep="\t", header=True, index=False)
if verbose:
    model_probs_df.head()

In [ ]:
foobar baz

In [ ]:
# take the first fold of the k-fold split
# train_index, test_index = group_kfold.split(transformed_X, y, group_by_admsn).next()
# X_train, X_test = transformed_X[train_index], transformed_X[test_index]
# y_train, y_test = y[train_index], y[test_index]

# ddd = {"OR_CASE_ID": or_case_id_number,
#        "ADMSN_ID": admsn_ids, 
#        "ADMSN_SURGERY_NUMBER": admsn_surgery_number,
#        "INPT_DEATH_YN": y}

cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
model_probs_cv = []
model_count = 0
for model in models_cv:
    print("starting model", model_count)
    probs = cross_val_predict(model, df.iloc[:,0:], y, 
                      cv=group_kfold.split(df.iloc[:,0:], y, group_by_admsn), 
                      n_jobs=1, method='predict_proba', verbose=3) 
    print("finished model", model_count)
#     ddd[model_names[model_count]] = model_probs_cv[model_count][:,1]
#     new_df = pd.DataFrame(data=ddd)
#     new_df.to_csv(os.path.join(dir_to_save_files, "model_pred_probs.txt"), sep="\t", header=True, index=False)
    model_count += 1
    model_probs_cv.append(probs)

#model_probs_cv = [cross_val_predict(model, df.iloc[:,0:], y, cv=group_kfold.split(df.iloc[:,0:], y, group_by_admsn), n_jobs=5, method='predict_proba') for model in models_cv]

In [ ]:
# ddd = {"ADMSN_ID": admsn_ids, 
#        "PRED_DEATH_LOGREG": model_probs_cv[0][:,1], 
#        "PRED_DEATH_ELASTICNET": model_probs_cv[1][:,1], 
#        "PRED_DEATH_RF": model_probs_cv[2][:,1],
#        "PRED_DEATH_XGB": model_probs_cv[3][:,1],
#        "INPT_DEATH_YN": y}
ddd = {"OR_CASE_ID": or_case_id_number,
       "ADMSN_ID": admsn_ids, 
       "ADMSN_SURGERY_NUMBER": admsn_surgery_number,
       "INPT_DEATH_YN": y}
for i in range(len(model_names)):
    ddd[model_names[i]] = model_probs_cv[i][:,1]
new_df = pd.DataFrame(data=ddd)
new_df.to_csv(os.path.join(dir_to_save_files, "model_pred_probs.txt"), sep="\t", header=True, index=False)

In [ ]:
# new_df.sort_values(by=["ADMSN_ID", "ADMSN_SURGERY_NUMBER"], inplace=True)
# new_df.head(5)
# print new_df.shape

In [ ]:
# print admsn_ids.shape
# print transformed_X.shape
# lldf = pd.DataFrame(data=np.concatenate((np.expand_dims(admsn_ids, axis=1), transformed_X), axis=1), columns=["ADMSN_ID"] + list(df.columns.values))

In [ ]:
# total_admissions = new_df["ADMSN_ID"].nunique()
# num_mult_admission = 0
# num_live_increase_risk_2nd_surgery = 0
# num_live_increase_risk_last_surgery = 0

# num_death = 0
# num_death_increase_risk_2nd_surgery = 0
# num_death_increase_risk_last_surgery = 0

# for name,group in new_df.groupby("ADMSN_ID"):
# #     print name[0:10]
# #     print group.shape
#     # if we have patients with multiple surgeries
#     if group.shape[0] > 1:
#         num_mult_admission += 1
#         #print group
#         if group["INPT_DEATH_YN"].iloc[0] == False:
#             # check if risk increased from first to second surgery
#             if group["PRED_DEATH_RF"].iloc[1] > group["PRED_DEATH_RF"].iloc[0]:
#                 num_live_increase_risk_2nd_surgery += 1
#             # check if risk increased from first to last surgery
#             if group["PRED_DEATH_RF"].iloc[-1] > group["PRED_DEATH_RF"].iloc[0]:
#                 num_live_increase_risk_last_surgery += 1
#         else:
#             num_death += 1
#             # check if risk increased from first to second surgery
#             if group["PRED_DEATH_RF"].iloc[1] > group["PRED_DEATH_RF"].iloc[0]:
#                 num_death_increase_risk_2nd_surgery += 1
#             # check if risk increased from first to last surgery
#             if group["PRED_DEATH_RF"].iloc[-1] > group["PRED_DEATH_RF"].iloc[0]:
#                 num_death_increase_risk_last_surgery += 1
            
# print "total admissions:", total_admissions
# print "number of deaths: {} ({}%)".format(num_death, num_death * 100. / total_admissions)
# print "number of survivors that increased in risk from 1st to 2nd surgery: {} ({}%)".format(
#     num_live_increase_risk_2nd_surgery, num_live_increase_risk_2nd_surgery * 100. / (num_mult_admission-num_death))
# print "number of survivors that increased in risk from 1st to LAST surgery: {} ({}%)".format(
#     num_live_increase_risk_last_surgery, num_live_increase_risk_last_surgery * 100. / (num_mult_admission-num_death))
# print "number of deaths that increased in risk from 1st to 2nd surgery: {} ({}%)".format(
#     num_death_increase_risk_2nd_surgery, num_death_increase_risk_2nd_surgery * 100. / num_death)
# print "number of deaths that increased in risk from 1st to LAST surgery: {} ({}%)".format(
#     num_death_increase_risk_last_surgery, num_death_increase_risk_last_surgery * 100. / num_death)

# i = 0
# plt.hold(True)
# for name,group in new_df.groupby("ADMSN_ID"):
# #     print name[0:10]
# #     print group.shape
#     # if we have patients with multiple surgeries
#     if group.shape[0] > 1:
#         #print group
#         plot_color = 'b' if group["INPT_DEATH_YN"].iloc[0] == False else 'r'
#         if group["INPT_DEATH_YN"].iloc[0] == False:
#             plt.plot(range(group.shape[0]), group["PRED_DEATH_RF"], plot_color)
#             i = i+1
#         if i > 100: 
#             break
# i = 0
# for name,group in new_df.groupby("ADMSN_ID"):
# #     print name[0:10]
# #     print group.shape
#     # if we have patients with multiple surgeries
#     if group.shape[0] > 1:
#         #print group
#         plot_color = 'b' if group["INPT_DEATH_YN"].iloc[0] == False else 'r'
#         if group["INPT_DEATH_YN"].iloc[0] == True:
#             plt.plot(range(group.shape[0]), group["PRED_DEATH_RF"], plot_color)
#             i = i+1
#         if i > 100: 
#             break
# plt.title("Change in mortality prediction over multiple surgeries")
# plt.xlabel("Surgery Number")
# plt.ylabel("P(death)")
# plt.show()

In [ ]:
plot_roc_curve(models, model_names, model_probs_cv, y, os.path.join(dir_to_save_files, "roc_curve_cv.tif"))

In [ ]:
plot_precision_recall_curve(models, model_names, model_probs_cv, y, os.path.join(dir_to_save_files, "precision_recall_cv.tif"))

In [ ]:
print "Brier Score Loss:"
with open(os.path.join(dir_to_save_files, "brier_score_cv.txt"), "w") as text_file:
    for i in range(len(models)):
        brier_loss = brier_score_loss(y, model_probs_cv[i][:, 1])
        print model_names[i],"\t\t", brier_loss
        text_file.write("{},{}\n".format(model_names[i], brier_loss))

In [ ]:
num_folds = 5

# group_by_admsn = admsn_ids.astype('category').cat.codes
# print "number of operations:", len(group_by_admsn)
# print "number of unique admission groups:", len(np.unique(group_by_admsn))
# group_kfold = GroupKFold(n_splits=num_folds)

#cv = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

def tp(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[1, 1]
def tn(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return metrics.confusion_matrix(y_true, y_pred)[1, 0]

scoring = {'tp' : make_scorer(tp), 'tn' : make_scorer(tn),
           'fp' : make_scorer(fp), 'fn' : make_scorer(fn),
            'accuracy': 'accuracy', 'roc_auc': 'roc_auc',
             'average_precision': 'average_precision',
          'precision': 'precision', 'recall': 'recall',
          'f1_score': 'f1'}

#scoring = ['accuracy', 'roc_auc', 'average_precision']
# model_scores_cv = [cross_validate(model, df.iloc[:,0:], y, scoring=scoring, 
#                                   cv=group_kfold.split(df.iloc[:,0:], y, group_by_admsn), 
#                                   n_jobs=1, verbose=2) for model in models_cv]

model_scores_cv = []
model_count = 0
for model in models_cv:
    print("starting model", model_count)
#     print(model.named_steps)
    mod_scores = cross_validate(model, df.iloc[:,0:], y, scoring=scoring, 
                                  cv=group_kfold.split(df.iloc[:,0:], y, group_by_admsn), 
                                  n_jobs=1, verbose=10)
    print("finished model", model_count)
    model_count += 1
    model_scores_cv.append(mod_scores)

# model_accuracy_cv = [cross_val_score(model, X_train, y_train, 
#                                    scoring='accuracy', cv=cv, n_jobs=-1) for model in models]
# model_roc_auc_cv = [cross_val_score(model, X_train, y_train, 
#                                    scoring='roc_auc', cv=cv, n_jobs=-1) for model in models]
# model_avg_precision_cv = [cross_val_score(model, X_train, y_train, 
#                                    scoring='average_precision', cv=cv, n_jobs=-1) for model in models]
print model_scores_cv

In [ ]:
fig, ax = plt.subplots(figsize=default_size)
index = np.arange(len(models))
bar_width = 0.35
opacity = 0.8
 
rects1 = plt.bar(index, [m['test_accuracy'].mean() for m in model_scores_cv], bar_width,
                 alpha=opacity,
                 color=PLOT_COLOURS[0],
                 yerr=[np.std(m['test_accuracy']) for m in model_scores_cv],
                 label='Mean Accuracy')
 
rects2 = plt.bar(index + bar_width, [m['test_roc_auc'].mean() for m in model_scores_cv], bar_width,
                 alpha=opacity,
                 color=PLOT_COLOURS[1],
                 yerr=[np.std(m['test_roc_auc']) for m in model_scores_cv],
                 label='Mean ROC AUC')

rects3 = plt.bar(index + 2*bar_width, [m['test_average_precision'].mean() for m in model_scores_cv], bar_width,
                 alpha=opacity,
                 color=PLOT_COLOURS[2],
                 yerr=[np.std(m['test_average_precision']) for m in model_scores_cv],
                 label='Mean Avg. Precision')

 
plt.xlabel('Model')
plt.ylabel('Score')
plt.title('Model Accuracy/ROC AUC/Avg. Precision with ASA_STATUS (CV=%d)' % num_folds)
plt.xticks(index + bar_width, model_names)
plt.ylim([0.0, 1.05])
plt.yticks(np.arange(0., 1.1, 0.05))
plt.legend(loc="lower right")
plt.gca().yaxis.grid(True)
plt.plot([1.0 - y_test.mean()]*(len(models)), color='red', lw=2, linestyle='--')
plt.tight_layout()
plt.savefig(os.path.join(dir_to_save_files, "accuracy_roc_auc_cv.tif"), format="tif", dpi=fig_dpi)
plt.show()

In [ ]:
def ci(results):
    return st.t.interval(0.95, len(results)-1, loc=np.mean(results), scale=st.sem(results))

In [ ]:
with open(os.path.join(dir_to_save_files, "model_scores_cv.txt"), "w") as text_file:
    text_file.write("Model," + ",".join(scoring.keys()) + ",specificity" + "\n")
    for mod in model_names:
        print "Model:", mod
        text_file.write(mod + ",")
        for score in scoring.keys():
            score_vals = model_scores_cv[model_names.index(mod)]['test_' + score]
            print "\t", score, "\t{:0.3f} ({:0.3f}-{:0.3f})".format(score_vals.mean(), ci(score_vals)[0], ci(score_vals)[1])
            text_file.write("{:0.3f} ({:0.3f}-{:0.3f}),".format(score_vals.mean(), ci(score_vals)[0], ci(score_vals)[1]))
            #print "mod:", mod, "score:", score

        tn = model_scores_cv[model_names.index(mod)]['test_tn']
        fp = model_scores_cv[model_names.index(mod)]['test_fp']
        specificity_vals = np.array(tn, dtype="float") / (np.array(tn) + np.array(fp))
        print "\tspecificity:\t", "\t{:0.3f} ({:0.3f}-{:0.3f})".format(specificity_vals.mean(), ci(specificity_vals)[0], ci(specificity_vals)[1])
        text_file.write("{:0.3f} ({:0.3f}-{:0.3f})\n".format(specificity_vals.mean(), ci(specificity_vals)[0], ci(specificity_vals)[1]))
        precision_vals = score_vals = model_scores_cv[model_names.index(mod)]['test_precision']
        recall_vals = score_vals = model_scores_cv[model_names.index(mod)]['test_recall']
        f1_score_vals = np.nan_to_num(2.0*(precision_vals*recall_vals)/(precision_vals + recall_vals))
        print "\tf1 score:\t", "\t{:0.3f} ({:0.3f}-{:0.3f})".format(f1_score_vals.mean(), ci(f1_score_vals)[0], ci(f1_score_vals)[1])
        #text_file.write("\t{:0.3f} ({:0.3f}-{:0.3f})\n".format(f1_score_vals.mean(), ci(f1_score_vals)[0], ci(f1_score_vals)[1]))

In [ ]:
rf_cv_auc = model_scores_cv[model_names.index('Random Forest')]['test_roc_auc']
print "Random Forest CV AUC", rf_cv_auc
print rf_cv_auc.mean()
print ci(rf_cv_auc)
rf_cv_precision = model_scores_cv[model_names.index('Random Forest')]['test_precision']
print "Random Forest CV Precision", rf_cv_precision
print rf_cv_precision.mean()
print ci(rf_cv_precision)

In [ ]:
en_cv_auc = model_scores_cv[model_names.index('ElasticNet')]['test_roc_auc']
print "ElasticNet CV AUC", en_cv_auc
print en_cv_auc.mean()
print ci(en_cv_auc)

In [ ]:
lr_cv_auc = model_scores_cv[model_names.index('Log. Regression')]['test_roc_auc']
print "Log. Regression CV AUC", lr_cv_auc
print lr_cv_auc.mean()
print ci(lr_cv_auc)

In [ ]:
lr_cv_auc = model_scores_cv[model_names.index('XGBClassifier')]['test_roc_auc']
print "XGBClassifier CV AUC", lr_cv_auc
print lr_cv_auc.mean()
print ci(lr_cv_auc)

In [ ]:
######################################################
# Code for imputing ASA status
######################################################

#scaler = StandardScaler() 
scaler = StandardizeWithNaN()
imp = SoftImputeEstimator(verbose=True)
smt = SMOTE(kind='borderline1', k_neighbors=3)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#rfr = RandomForestRegressor(oob_score=True, n_estimators=2000, n_jobs=1, max_features="log2")
rfr = XGBRegressor(max_depth=7, n_estimators=2000, n_jobs=-1)
rfr_pipe = Pipeline([('scaler', scaler), ('impute', imp), ('smt', smt), ('rfr', rfr)])
asa_predictions = cross_val_predict(rfr_pipe, df.iloc[:,0:], y, cv=group_kfold.split(df.iloc[:,0:], y, group_by_admsn), n_jobs=1)
#xgb = XGBClassifier(max_depth=5, objective="binary:logistic", n_estimators=2000, n_jobs=-1)
#xgb_pipe = Pipeline([('smt', smt), ('scaler', scaler), ('rfr', xgb)])
#asa_predictions = cross_val_predict(xgb_pipe, transformed_X, y, cv=cv, n_jobs=5, method='predict_proba')

In [ ]:
print "Number of predictions:", len(asa_predictions)
print df.shape
print asa_predictions[0:10]

In [ ]:
print y[0:10]

In [ ]:
input_death_yn[0:10]
death_indices = [i for i,x in enumerate(input_death_yn) if x]
live_indices = [i for i,x in enumerate(input_death_yn) if not x]
print death_indices[0:10]
print len(death_indices)
print len(live_indices)

In [ ]:
print asa_predictions[death_indices]
print y[death_indices]

In [ ]:
plt.scatter(y[death_indices], asa_predictions[death_indices])
plt.xlabel("True ASA_STATUS")
plt.ylabel("Predicted ASA_STATUS")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print mean_squared_error(y[death_indices], asa_predictions[death_indices])
print r2_score(y[death_indices], asa_predictions[death_indices])

In [ ]:
plt.scatter(y[live_indices], asa_predictions[live_indices])
plt.xlabel("True ASA_STATUS")
plt.ylabel("Predicted ASA_STATUS")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print mean_squared_error(y[live_indices], asa_predictions[live_indices])
print r2_score(y[live_indices], asa_predictions[live_indices])

In [ ]:
ddd = {"ADMSN_ID": admsn_ids, "OR_CASE_ID": or_case_id_number, "PRED_ASA_STATUS": asa_predictions, "ASA_STATUS": y}
#ddd = {"ADMSN_ID": admsn_ids, "PRED_DEATH": asa_predictions[:,1], "INPT_DEATH_YN": y}
#ddd = {"ADMSN_ID": admsn_ids, "PRED_AKIN_EVENT": model_probs_cv[2][:,1], "AKIN_EVENT": y}
#ddd = {"ADMSN_ID": admsn_ids, "PRED_DEATH": model_probs_cv[2][:,1], "INPT_DEATH_YN": y}

In [ ]:
print len(y)
print len(admsn_ids)
#print len(model_probs_cv[2][:,1])
new_df = pd.DataFrame(data=ddd)
print df.shape

#print len(asa_predictions)

In [ ]:
new_df.boxplot(column="PRED_ASA_STATUS", by="ASA_STATUS")
plt.title("")
plt.suptitle("")
plt.ylabel("PRED_ASA_STATUS")

In [ ]:
new_df.head(30)

In [ ]:
new_df.to_csv(os.path.join(dir_to_save_files, "ASA_predictions_noTime.txt"), sep="\t", header=True, index=False)
if False:
    #pass
    new_df.to_csv("AKIN_EVENT_predictions.txt", sep="\t", header=True, index=False)

In [ ]:
new_df.describe(include="all")